# UMGC Catalog
## Step 4: Load additional data into UMGC.db

In [1]:
import sqlite3
conn = sqlite3.connect('wave/UMGC.db')
c = conn.cursor()

import rawdata as r

#import database_table_source as dts

def drop_table(table, db=c):
    db.execute('DROP TABLE IF EXISTS ' + table)

def drop_view(table, db=c):
    db.execute('DROP VIEW IF EXISTS ' + table)

def column_exists(table_name, column_name, db=c):
    # Get the info of all columns of the table
    db.execute(f"PRAGMA table_info({table_name})")
    columns = db.fetchall()

    # Check if the column exists in the table
    for column in columns:
        if column[1] == column_name:
            return True

    return False

In [2]:
###############################################################
# Catalog Table

catalog = [
    { "id":  1, "version": "2023-2024" }
]

drop_table('catalog',c)
c.execute('''
    CREATE TABLE catalog (
        id INTEGER PRIMARY KEY,
        version TEXT
    )
''')

c.executemany('INSERT INTO catalog VALUES (:id, :version)', catalog)

# Update classes to contain catalog version
# Use the function to check if 'catalog_id' exists in 'classes'
if not column_exists('classes', 'catalog_id'):
    c.execute('ALTER TABLE classes ADD COLUMN catalog_id INTEGER')
    c.execute('UPDATE classes SET catalog_id = 1 WHERE catalog_id IS NULL')

conn.commit()

In [3]:
################################################################
# Degrees Table

degrees = [
    { "id":  1, "name": "AA", "type": "Associate" },
    { "id":  2, "name": "BA", "type": "Bachelor's" },
    { "id":  3, "name": "BS", "type": "Bachelor's" },
    { "id":  4, "name": "BSN", "type": "Bachelor's" },
    { "id":  5, "name": "Minor", "type": "Bachelor's Minor" },
    { "id":  6, "name": "MA", "type": "Master's" },
    { "id":  7, "name": "MAT", "type": "Master's" },
    { "id":  8, "name": "MBA", "type": "Master's" },
    { "id":  9, "name": "MDE", "type": "Master's" },
    { "id": 10, "name": "MEd", "type": "Master's" },
    { "id": 11, "name": "MS", "type": "Master's" },
    { "id": 12, "name": "DBA", "type": "Doctorate" },
    { "id": 13, "name": "DM", "type": "Doctorate" },
    { "id": 14, "name": "UC", "type": "Undergraduate Certificate" },
    { "id": 15, "name": "GC", "type": "Graduate Certificate "}
]

drop_table('degrees',c)
c.execute('''
    CREATE TABLE degrees (
        id INTEGER PRIMARY KEY,
        name TEXT,
        type TEXT
    )
''')

c.executemany('INSERT INTO degrees VALUES (:id, :name, :type)', degrees)
#c.executemany('INSERT INTO degrees VALUES (:id, :name, :type)', dts.degrees)

conn.commit()

In [4]:
################################################################
# Programs Table

programs = [
    { "id":   1, "degree_id":  1, "name": "General Studies" },
    { "id":   2, "degree_id":  3, "name": "Accounting" },
    { "id":   3, "degree_id":  3, "name": "Applied Technology" },
    { "id":   4, "degree_id":  3, "name": "Biotechnology" },
    { "id":   5, "degree_id":  3, "name": "Business Administration" },
    { "id":   6, "degree_id":  2, "name": "Communication Studies" },
    { "id":   7, "degree_id":  3, "name": "Computer Science" },
    { "id":   8, "degree_id":  3, "name": "Criminal Justice" },
    { "id":   9, "degree_id":  3, "name": "Cybersecurity Management and Policy" },
    { "id":  10, "degree_id":  3, "name": "Cybersecurity Technology" },
    { "id":  11, "degree_id":  3, "name": "Data Science" },
    { "id":  12, "degree_id":  2, "name": "East Asian Studies" },
    { "id":  13, "degree_id":  2, "name": "English" },
    { "id":  14, "degree_id":  3, "name": "Environmental Health and Safety " },
    { "id":  15, "degree_id":  3, "name": "Finance" },
    { "id":  16, "degree_id":  3, "name": "General Studies" },
    { "id":  17, "degree_id":  3, "name": "Gerontology and Aging Services" },
    { "id":  18, "degree_id":  2, "name": "Graphic Communication" },
    { "id":  19, "degree_id":  3, "name": "Health Services Management" },
    { "id":  20, "degree_id":  2, "name": "History" },
    { "id":  21, "degree_id":  3, "name": "Homeland Security" },
    { "id":  22, "degree_id":  3, "name": "Human Resource Management" },
    { "id":  23, "degree_id":  2, "name": "Humanities" },
    { "id":  24, "degree_id":  3, "name": "Laboratory Management" },
    { "id":  25, "degree_id":  3, "name": "Legal Studies" },
    { "id":  26, "degree_id":  3, "name": "Management Information Systems" },
    { "id":  27, "degree_id":  3, "name": "Management Studies" },
    { "id":  28, "degree_id":  3, "name": "Marketing" },
    { "id":  29, "degree_id":  4, "name": "Nursing for Registered Nurses" },
    { "id":  30, "degree_id":  3, "name": "Political Science" },
    { "id":  31, "degree_id":  3, "name": "Psychology" },
    { "id":  32, "degree_id":  3, "name": "Public Safety Administration" },
    { "id":  33, "degree_id":  3, "name": "Social Science" },
    { "id":  34, "degree_id":  3, "name": "Software Development and Security" },
    { "id":  35, "degree_id":  3, "name": "Web and Digital Design" },
    { "id":  36, "degree_id":  5, "name": "Accounting" },
    { "id":  37, "degree_id":  5, "name": "African American Studies" },
    { "id":  38, "degree_id":  5, "name": "Art" },
    { "id":  39, "degree_id":  5, "name": "Art History" },
    { "id":  40, "degree_id":  5, "name": "Biology" },
    { "id":  41, "degree_id":  5, "name": "Business Administration" },
    { "id":  42, "degree_id":  5, "name": "Communication Studies" },
    { "id":  43, "degree_id":  5, "name": "Computer Science" },
    { "id":  44, "degree_id":  5, "name": "Criminal Justice" },
    { "id":  45, "degree_id":  5, "name": "Cybersecurity" },
    { "id":  46, "degree_id":  5, "name": "Data Science" },
    { "id":  47, "degree_id":  5, "name": "Diversity Awareness" },
    { "id":  48, "degree_id":  5, "name": "East Asian Studies" },
    { "id":  49, "degree_id":  5, "name": "Economics" },
    { "id":  50, "degree_id":  5, "name": "Emergency Management" },
    { "id":  51, "degree_id":  5, "name": "English" },
    { "id":  52, "degree_id":  5, "name": "Environmental Health and Safety" },
    { "id":  53, "degree_id":  5, "name": "Finance" },
    { "id":  54, "degree_id":  5, "name": "Fire Service Administration" },
    { "id":  55, "degree_id":  5, "name": "Forensics" },
    { "id":  56, "degree_id":  5, "name": "Gerontology and Aging Services" },
    { "id":  57, "degree_id":  5, "name": "Health Services Management" },
    { "id":  58, "degree_id":  5, "name": "History" },
    { "id":  59, "degree_id":  5, "name": "Homeland Security" },
    { "id":  60, "degree_id":  5, "name": "Human Resource Management" },
    { "id":  61, "degree_id":  5, "name": "Law for Business" },
    { "id":  62, "degree_id":  5, "name": "Management Information Systems" },
    { "id":  63, "degree_id":  5, "name": "Marketing" },
    { "id":  64, "degree_id":  5, "name": "Mathematical Sciences" },
    { "id":  65, "degree_id":  5, "name": "Natural Science" },
    { "id":  66, "degree_id":  5, "name": "Personal Financial Planning" },
    { "id":  67, "degree_id":  5, "name": "Philosophy" },
    { "id":  68, "degree_id":  5, "name": "Political Science" },
    { "id":  69, "degree_id":  5, "name": "Psychology" },
    { "id":  70, "degree_id":  5, "name": "Public Safety Administration" },
    { "id":  71, "degree_id":  5, "name": "Small Business Management and Entrepreneurship" },
    { "id":  72, "degree_id":  5, "name": "Sociology" },
    { "id":  73, "degree_id":  5, "name": "Speech Communication" },
    { "id":  74, "degree_id":  5, "name": "Terrorism and Critical Infrastructure" },
    { "id":  75, "degree_id":  5, "name": "Web and Digital Design" },
    { "id":  76, "degree_id":  5, "name": "Women, Gender, and Sexuality Studies" },
    { "id":  77, "degree_id": 11, "name": "Accounting and Financial Management" },
    { "id":  78, "degree_id": 11, "name": "Acquisition and Contract Management" },
    { "id":  79, "degree_id": 11, "name": "Biotechnology: Bioinformatics" },
    { "id":  80, "degree_id": 11, "name": "Biotechnology: Biosecurity and Biodefense" },
    { "id":  81, "degree_id": 11, "name": "Biotechnology: Biotechnology Management" },
    { "id":  82, "degree_id": 11, "name": "Biotechnology: Biotechnology Regulatory Affairs" },
    { "id":  83, "degree_id":  8, "name": "Business Administration" },
    { "id":  84, "degree_id": 11, "name": "Cloud Computing Systems" },
    { "id":  85, "degree_id": 11, "name": "CyberAccounting" },
    { "id":  86, "degree_id": 11, "name": "Cyber Operations" },
    { "id":  87, "degree_id": 11, "name": "Cybersecurity Management and Policy" },
    { "id":  88, "degree_id": 11, "name": "Cybersecurity Technology" },
    { "id":  89, "degree_id": 11, "name": "Data Analytics" },
    { "id":  90, "degree_id": 11, "name": "Digital Forensics and Cyber Investigation" },
    { "id":  91, "degree_id":  9, "name": "Distance Education and E-Learning" },
    { "id":  92, "degree_id": 11, "name": "Environmental Management" },
    { "id":  93, "degree_id": 11, "name": "Healthcare Administration" },
    { "id":  94, "degree_id": 11, "name": "Health Information Management and Technology" },
    { "id":  95, "degree_id": 11, "name": "Information Technology: Database Systems Technology" },
    { "id":  96, "degree_id": 11, "name": "Information Technology: Homeland Security Management" },
    { "id":  97, "degree_id": 11, "name": "Information Technology: Informatics" },
    { "id":  98, "degree_id": 11, "name": "Information Technology: Information Assurance" },
    { "id":  99, "degree_id": 11, "name": "Information Technology: Project Management" },
    { "id": 100, "degree_id": 11, "name": "Information Technology: Software Engineering" },
    { "id": 101, "degree_id": 11, "name": "Information Technology: Systems Engineering" },
    { "id": 102, "degree_id": 10, "name": "Instructional Technology" },
    { "id": 103, "degree_id": 11, "name": "Learning Design and Technology" },
    { "id": 104, "degree_id": 11, "name": "Management: Accounting" },
    { "id": 105, "degree_id": 11, "name": "Management: Criminal Justice Management" },
    { "id": 106, "degree_id": 11, "name": "Management: Emergency Management" },
    { "id": 107, "degree_id": 11, "name": "Management: Financial Management" },
    { "id": 108, "degree_id": 11, "name": "Management: Homeland Security Management" },
    { "id": 109, "degree_id": 11, "name": "Management: Human Resource Management" },
    { "id": 110, "degree_id": 11, "name": "Management: Information Systems and Services" },
    { "id": 111, "degree_id": 11, "name": "Management: Intelligence Management" },
    { "id": 112, "degree_id": 11, "name": "Management: Interdisciplinary Studies in Management" },
    { "id": 113, "degree_id": 11, "name": "Management: Marketing" },
    { "id": 114, "degree_id": 11, "name": "Management: Nonprofit and Association Management" },
    { "id": 115, "degree_id": 11, "name": "Management: Project Management" },
    { "id": 116, "degree_id": 11, "name": "Strategic Communications" },
    { "id": 117, "degree_id":  7, "name": "Teaching" },
    { "id": 118, "degree_id": 11, "name": "Transformational Leadership" },
    { "id": 119, "degree_id": 12, "name": "Business Administration" },
    { "id": 120, "degree_id": 13, "name": "Management: Community College Policy and Administration" },
    { "id": 121, "degree_id": 14, "name": "Accounting Foundations" },
    { "id": 122, "degree_id": 14, "name": "Advanced Management" },
    { "id": 123, "degree_id": 14, "name": "American Government and Political Processes" },
    { "id": 124, "degree_id": 14, "name": "Applied Social Sciences" },
    { "id": 125, "degree_id": 14, "name": "Augmented and Virtual Reality Design" },
    { "id": 126, "degree_id": 14, "name": "Clinical Mental Health Care" },
    { "id": 127, "degree_id": 14, "name": "Computer Networking" },
    { "id": 128, "degree_id": 14, "name": "Cyber Threat Hunting" },
    { "id": 129, "degree_id": 14, "name": "Data Analytics" },
    { "id": 130, "degree_id": 14, "name": "Decision Support for Business" },
    { "id": 131, "degree_id": 14, "name": "Digital Design" },
    { "id": 132, "degree_id": 14, "name": "Digital Marketing" },
    { "id": 133, "degree_id": 14, "name": "Health Information Management and Data Analytics" },
    { "id": 134, "degree_id": 14, "name": "HR People Analytics" },
    { "id": 135, "degree_id": 14, "name": "Human Resource Management" },
    { "id": 136, "degree_id": 14, "name": "Leadership and Ethics" },
    { "id": 137, "degree_id": 14, "name": "Machine Learning" },
    { "id": 138, "degree_id": 14, "name": "Management" },
    { "id": 139, "degree_id": 14, "name": "Management Information Systems" },
    { "id": 140, "degree_id": 14, "name": "Project Management" },
    { "id": 141, "degree_id": 14, "name": "Public Safety Executive Leadership" },
    { "id": 142, "degree_id": 14, "name": "Spanish for Business and the Professions" },
    { "id": 143, "degree_id": 14, "name": "Vulnerability Assessment" },
    { "id": 144, "degree_id": 14, "name": "Watershed Management" },
    { "id": 145, "degree_id": 14, "name": "Web Design" },
    { "id": 146, "degree_id": 14, "name": "Women, Gender, and Sexuality Studies" },
    { "id": 147, "degree_id": 15, "name": "Accounting Information Security" },
    { "id": 148, "degree_id": 15, "name": "Acquisition and Contract Management" },
    { "id": 149, "degree_id": 15, "name": "Bioinformatics" },
    { "id": 150, "degree_id": 15, "name": "Business Analytics" },
    { "id": 151, "degree_id": 15, "name": "Cloud Computing and Networking" },
    { "id": 152, "degree_id": 15, "name": "Cyber Operations" },
    { "id": 153, "degree_id": 15, "name": "Cybersecurity Management and Policy" },
    { "id": 154, "degree_id": 15, "name": "Cybersecurity Technology" },
    { "id": 155, "degree_id": 15, "name": "Digital Forensics and Cyber Investigation" },
    { "id": 156, "degree_id": 15, "name": "Digital Health Leader" },
    { "id": 157, "degree_id": 15, "name": "Global Health Management" },
    { "id": 158, "degree_id": 15, "name": "Homeland Security Management" },
    { "id": 159, "degree_id": 15, "name": "Informatics" },
    { "id": 160, "degree_id": 15, "name": "Information Assurance" },
    { "id": 161, "degree_id": 15, "name": "Instructional Technology Integration" },
    { "id": 162, "degree_id": 15, "name": "Leadership and Management" },
    { "id": 163, "degree_id": 15, "name": "Learning Design and Technology" },
    { "id": 164, "degree_id": 15, "name": "Long-Term Care Administration" },
    { "id": 165, "degree_id": 15, "name": "Multicultural Marketing" },
    { "id": 166, "degree_id": 15, "name": "Project Management" },
    { "id": 167, "degree_id": 15, "name": "Strategic Communications" },
    { "id": 168, "degree_id": 15, "name": "Strategic Human Resource Management" },
    { "id": 169, "degree_id": 15, "name": "Systems Engineering" }
]

drop_table('programs',c)
c.execute('''
    CREATE TABLE programs (
        id INTEGER PRIMARY KEY,
        name TEXT,
        degree_id INTEGER
    )
''')

c.executemany('INSERT INTO programs VALUES (:id, :name, :degree_id)', programs)

# Update programs to contain catalog version
c.execute('ALTER TABLE programs ADD catalog_id INTEGER')
c.execute('UPDATE programs SET catalog_id = 1')
conn.commit()

In [5]:
################################################################
# "Course Type" Table

course_type = [
    { "id":  1, "name": "major", "label": "Major" },
    { "id":  2, "name": "required", "label": "Required" },
    { "id":  3, "name": "general", "label": "General" },
    { "id":  4, "name": "elective", "label": "Elective" },
    { "id":  5, "name": "related", "label": "Related" },    
    { "id":  6, "name": "required_ge", "label": "Required,General" },
    { "id":  7, "name": "required_elective", "label": "Required,Elective" }
]

drop_table('course_type',c)
c.execute('''
    CREATE TABLE course_type (
        id INTEGER PRIMARY KEY,
        name TEXT,
        label TEXT
    )
''')

c.executemany('INSERT INTO course_type VALUES (:id, :name, :label)', course_type)
conn.commit()

In [6]:
################################################################
# "General Education Requirements" Table

drop_table('general_education_requirements',c)
c.execute('''
    CREATE TABLE general_education_requirements (
        id INTEGER PRIMARY KEY,
        requirement INTEGER,
        type TEXT,
        part TEXT DEFAULT '',
        description TEXT,
        credits INTEGER,
        note TEXT DEFAULT ''
    )
''')

################################################################
# "General Education" Table

drop_table('general_education',c)
c.execute('''
    CREATE TABLE general_education (
        id INTEGER PRIMARY KEY,
        general_education_requirements_id INTEGER,
        course_id INTEGER,
        name TEXT,
        note TEXT DEFAULT '',
        FOREIGN KEY(general_education_requirements_id) 
            REFERENCES general_education_requirements(id),        
        FOREIGN KEY(course_id) REFERENCES classes(id)
    )
''')

# General Education Requirement 1
## Communications: 12 credits
## 1. WRTG 111 or another writing course (3 credits)

query = '''
    INSERT INTO general_education_requirements (id, requirement, type, part, description, credits)
    SELECT 1, 1, 'Communications', '1', 'WRTG 111 or another writing course', 3
'''
c.execute(query)

query = '''
INSERT INTO general_education (general_education_requirements_id, course_id, name)
    SELECT 
        1, id, name 
	FROM classes
	WHERE 
		credits = '3' 
		AND (
		    name LIKE 'WRTG %'  
			OR name IN ('COMM 390', 'COMM 492','ENGL 102','JOUR 201')
		)
		AND name NOT IN ('WRTG 112', 'WRTG 288', 'WRTG 388','WRTG 486%')
'''
c.execute(query)

## 2. WRTG 112
## Must be completed within the first 24 credits

query = '''
    INSERT INTO general_education_requirements (id, requirement, type, part, description, credits, note)
    SELECT 2, 1, 'Communications', '2', 'WRTG 112', 3, 'Must be completed within the first 24 credits'
'''
c.execute(query)

query = '''
INSERT INTO general_education (general_education_requirements_id, course_id, name)
    SELECT 
        2, id, name 
	FROM classes
	WHERE 
        name = 'WRTG 112'
'''
c.execute(query)

## 3. A course in communication, writing, or speech
## 

query = '''
    INSERT INTO general_education_requirements (id, requirement, type, part, description, credits)
    SELECT 3, 1, 'Communications', '3', 'A course in communication, writing, or speech', 3
'''
c.execute(query)

query = '''
INSERT INTO general_education (general_education_requirements_id, course_id, name)
    SELECT 
        3, id, name 
	FROM classes
	WHERE 
		credits = '3' 
		AND (
            name IN ('ENGL 102', 'ENGL 281', 'JOUR 201')
			OR name LIKE 'COMM %'
            OR name LIKE 'SPCH %'
            OR name LIKE 'WRTG %'
		)
		AND name NOT LIKE '% 486A'
		AND name NOT LIKE '% 486B'
		AND name NOT LIKE 'WRTG 111'
		AND name NOT LIKE 'WRTG 112'
'''
c.execute(query)

## 4. An upper-level advanced writing course
## 
query = '''
    INSERT INTO general_education_requirements (id, requirement, type, part, description, credits)
    SELECT 4, 1, 'Communications', '4', 'An upper-level advanced writing course', 3
'''
c.execute(query)

query = '''
INSERT INTO general_education (general_education_requirements_id, course_id, name)
    SELECT 
        4, id, name 
	FROM classes
	WHERE 
		credits = '3' 
		AND 
            name IN ('WRTG 391', 'WRTG 393','WRTG 394')
;
'''
c.execute(query)

## Mathematics: 3 credits
## Must be completed within the first 24 credits
query = '''
    INSERT INTO general_education_requirements (id, requirement, type, description, credits, note)
    SELECT 5, 2, 'Mathematics', 'Mathematics', 3, 'Must be completed within the first 24 credits'
'''
c.execute(query)

query = '''
INSERT INTO general_education (general_education_requirements_id, course_id, name)
    SELECT 
        5, id, name 
	FROM classes
	WHERE 
		name IN ('MATH 105', 'MATH 107', 'MATH 115', 'MATH 140', 'STAT 200')
'''
c.execute(query)

## Arts and Humanities: 6 credits
## Two 3-credit courses

query = '''
    INSERT INTO general_education_requirements (id, requirement, type, description, credits, note)
    SELECT 6, 3, 'Arts and Humanities', 'Arts and Humanities', 6, 'Two 3-credit courses'
'''
c.execute(query)

query = '''
INSERT INTO general_education (general_education_requirements_id, course_id, name)
    SELECT 
        6, id, name 
	FROM classes
	WHERE 
		credits = '3' 
		AND (
 		       name LIKE 'ARTH %'
 			OR name LIKE 'ARTT %'
 			OR name LIKE 'ASTD %' 
 			OR name LIKE 'ENGL %' 
 			OR name LIKE 'GRCO %' 
 			OR name LIKE 'HIST %'
 			OR name LIKE 'HUMN %' 
 			OR name LIKE 'MUSC %' 
 			OR name LIKE 'PHIL %' 
 			OR name LIKE 'THET %' 
 			OR name IN (
                    'ENGL 250', 
                    'ENGL 303', 
                    'ENGL 310',
                    'ENGL 430', 
                    'ENGL 363', 
                    'ENGL 364',
                    'ENGL 433',
                    'ENGL 441',
                    'ENGL 311',
                    'ENGL 312',
                    'ENGL 386', 
                    'ENGL 406',
                    'ENGL 459',
                    'ENGL 495' 
               ) 
 			OR name LIKE 'ARAB 11_'
 			OR name LIKE 'CHIN 11_'
 			OR name LIKE 'FREN 11_'
 			OR name LIKE 'GERM %'
               OR name LIKE 'JAPN 11_' 
               OR name LIKE 'JAPN 22_'
 			OR name IN (
                    'SPAN 111', 
                    'SPAN 112', 
                    'SPAN 211',
                    'SPAN 212', 
                    'SPAN 311', 
                    'SPAN 314'               
               ) 
          )
          AND name NOT IN ('ENGL 281', 'ENGL 384')
'''
c.execute(query)

# 7.
## Biological and Physical Sciences
## 1a. A science course combining lecture and laboratory (4 credits)
## 1b. A science course combining lecture and laboratory for science majors and minors (4 credits)
## 1c. A science lecture course (3 credits) with related laboratory course (1 credit)

query = '''
    INSERT INTO general_education_requirements (id, requirement, type, part, description, credits, note)
    VALUES 
        (7, 4, 'Biological and Physical Sciences', '1a', 'Science lecture with laboratory', 4, 
            'A science course combining lecture and laboratory (4 credits)'),
        (8, 4, 'Biological and Physical Sciences', '1b', 'Science lecture with laboratory for science majors or minors', 4, 
            'A science course combining lecture and laboratory (4 credits)'),
        (9, 4, 'Biological and Physical Sciences', '1c', 'Science lecture with laboratory', 4, 
            'A science lecture course (3 credits) with related laboratory course (1 credit)')
'''
c.execute(query)

query = '''
INSERT INTO general_education (general_education_requirements_id, course_id, name)
    SELECT 
        7, id, name 
	FROM classes
	WHERE 
		name in (
            'BIOL 103', 
            'BIOL 230',
			'NSCI 103'
        )
'''
c.execute(query)

query = '''
INSERT INTO general_education (general_education_requirements_id, course_id, name)
    SELECT 
        8, id, name 
	FROM classes
	WHERE 
		name in (
            'CHEM 103', 
            'PHYS 121',
			'PHYS 122'
        )
'''
c.execute(query)

query = '''
INSERT INTO general_education (general_education_requirements_id, course_id, name, note)
    SELECT 
        9, id, name, 'Pair: BIOL 101 (3) & BIOL 102 (1)' 
	FROM classes
	WHERE 
		name in (
            'BIOL 101',
            'BIOL 102'
        )
'''
c.execute(query)

query = '''
INSERT INTO general_education (general_education_requirements_id, course_id, name, note)
    SELECT 
        9, id, name, 'Pair: BIOL 160 (3) & BIOL 161 (1)' 
	FROM classes
	WHERE 
		name in (
            'BIOL 160',
            'BIOL 161'
        )
'''
c.execute(query)

query = '''
INSERT INTO general_education (general_education_requirements_id, course_id, name, note)
    SELECT 
        9, id, name, 'Pair: NSCI 100 (3) & NSCI 101 (1)' 
	FROM classes
	WHERE 
		name in (
            'NSCI 100',
            'NSCI 101'
        )
'''
c.execute(query)

query = '''
INSERT INTO general_education (general_education_requirements_id, course_id, name, note)
    SELECT 
        9, id, name, 'Pair: NSCI 170 (3) & NSCI 171 (1)' 
	FROM classes
	WHERE 
		name IN (
            'NSCI 170',
            'NSCI 171'
        )
'''
c.execute(query)

query = '''
INSERT INTO general_education (general_education_requirements_id, course_id, name, note)
    SELECT 
        9, id, name, 'Pair: NUTR 100 (3) & NUTR 101 (1)' 
	FROM classes
	WHERE 
		name in (
            'NUTR 100',
            'NUTR 101'
        )
'''
c.execute(query)

query = '''
INSERT INTO general_education (general_education_requirements_id, course_id, name, note)
    SELECT 
        9, id, name, '' 
	FROM classes
	WHERE 
		name in (
            'NSCI 120'
        )
'''
c.execute(query)

## 2. Any other science course (3 credits)
## Courses from the following disciplines apply: ASTR, BIOL, CHEM, GEOL, NSCI, NUTR, or PHYS. 

query = '''
    INSERT INTO general_education_requirements (id, requirement, type, part, description, credits, note)
    SELECT 10, 4, 'Biological and Physical Sciences', '2', 'Any other science course', 3, 
        'Courses from the following disciplines apply: ASTR, BIOL, CHEM, GEOL, NSCI, NUTR, or PHYS.'
'''
c.execute(query)

query = '''
INSERT INTO general_education (general_education_requirements_id, course_id, name)
    SELECT 
        10, id, name 
	FROM classes
	WHERE 
		credits = '3' 
		AND (
 		       name LIKE 'ASTR %'
 			OR name LIKE 'BIOL %'
 			OR name LIKE 'CHEM %'
 			OR name LIKE 'GEOL %'
 			OR name LIKE 'NSCI %'
 			OR name LIKE 'NUTR %'
 			OR name LIKE 'PHYS %'
        )
'''
c.execute(query)

# 8.
## Behavioral and Social Sciences: 6 credits
## Two 3-credit hour courses from the following:

query = '''
    INSERT INTO general_education_requirements (id, requirement, type, description, credits, note)
    VALUES 
        (11, 5, 'Behavioral and Social Sciences', 'Behavioral and Social Sciences', 6, 'Two 3-credit hour courses') 
'''
c.execute(query)

query = '''
INSERT INTO general_education (general_education_requirements_id, course_id, name)
    SELECT 
        11, id, name 
	FROM classes
	WHERE 
		credits = '3' 
		AND (
            name IN ('AASP 201', 'CCJS 100', 'CCJS 105', 'CCJS 350', 'CCJS 360', 'CCJS 461', 'WMST 200')
 			OR name LIKE 'ANTH %'
 			OR name LIKE 'ASTD %'
 			OR name LIKE 'BEHS %' 
 			OR name LIKE 'ECON %' 
 			OR name LIKE 'GEOG %' 
 			OR name LIKE 'GERO %'
 			OR name LIKE 'GVPT %' 
 			OR name LIKE 'PSYC %' 
 			OR name LIKE 'SOCY %')
        AND name NOT IN ('GERO 342', 'GERO 351')
        AND name NOT LIKE '% 486A'
'''
c.execute(query)

## Research and Computing Literacy: 7 credits

## Professional exploration course (3 credits)
## PACE 111B, PACE 111C, PACE 111M, PACE 111P, PACE 111S, and PACE 111T apply. To be taken as the first course.

query = '''
    INSERT INTO general_education_requirements (id, requirement, type, part, description, credits, note)
    VALUES 
        (12, 6, 'Research and Computing Literacy', '1', 'Professional exploration course', 3, 
            'To be taken as the first course'),
        (13, 6, 'Research and Computing Literacy', '2', 'Research skills and professional development course', 1, 
            'LIBS 150, CAPL 398A, and any general education course apply'),
        (14, 6, 'Research and Computing Literacy', '3', 'Computing or information technology course', 3, 
            'One 3-credit course or three 1-credit courses')
'''
c.execute(query)

query = '''
INSERT INTO general_education (general_education_requirements_id, course_id, name)
    SELECT 
        12, id, name 
	FROM classes
	WHERE 
		credits = '3' 
		AND name LIKE 'PACE 111_'
'''
c.execute(query)

query = '''
INSERT INTO general_education (general_education_requirements_id, course_id, name)
    SELECT 
        13, id, name 
	FROM classes
	WHERE 
		credits = '1' 
		AND name IN ('LIBS 150', 'CAPL 398A')
'''
c.execute(query)

query = '''
INSERT INTO general_education (general_education_requirements_id, course_id, name)
    SELECT 
        14, id, name 
	FROM classes
	WHERE 
		credits IN ('1', '3') 
		AND (
            name IN ('DATA 200', 'IFSM 201')
 			OR name LIKE 'CMIT %'
 			OR name LIKE 'CMSC %'
 			OR name LIKE 'CMST %' 
 			OR name LIKE 'CSIA %' 
 			OR name LIKE 'IFSM %' 
 			OR name LIKE 'SDEV %')
        AND name NOT LIKE '% 486A'
'''
c.execute(query)
conn.commit()

In [7]:
################################################################
# "Program Requirements" Table
# Bachelor's program requirement summary

program_requirements = [
    { "program_id":  2, "major": 36, "related_ge": 12, "related_elective": 15, "remaining_ge": 29, "remaining_elective": 28 },
    { "program_id":  3, "major": 30, "related_ge":  0, "related_elective":  0, "remaining_ge": 41, "remaining_elective": 49 },
    { "program_id":  4, "major": 36, "related_ge":  8, "related_elective":  9, "remaining_ge": 33, "remaining_elective": 34 },
    { "program_id":  5, "major": 33, "related_ge": 12, "related_elective":  0, "remaining_ge": 29, "remaining_elective": 46 },
    { "program_id":  6, "major": 33, "related_ge":  0, "related_elective":  0, "remaining_ge": 41, "remaining_elective": 46 },
    { "program_id":  7, "major": 36, "related_ge":  7, "related_elective":  7, "remaining_ge": 34, "remaining_elective": 36 },
    { "program_id":  8, "major": 33, "related_ge":  0, "related_elective":  0, "remaining_ge": 41, "remaining_elective": 46 },
    { "program_id":  9, "major": 33, "related_ge":  0, "related_elective":  0, "remaining_ge": 41, "remaining_elective": 46 },
    { "program_id": 10, "major": 33, "related_ge":  0, "related_elective":  0, "remaining_ge": 41, "remaining_elective": 46 },
    { "program_id": 11, "major": 39, "related_ge":  6, "related_elective":  0, "remaining_ge": 35, "remaining_elective": 40 },
    { "program_id": 12, "major": 30, "related_ge":  0, "related_elective":  0, "remaining_ge": 41, "remaining_elective": 49 },
    { "program_id": 13, "major": 33, "related_ge":  0, "related_elective":  0, "remaining_ge": 41, "remaining_elective": 46 },
    { "program_id": 14, "major": 36, "related_ge":  6, "related_elective":  0, "remaining_ge": 35, "remaining_elective": 43 },
    { "program_id": 15, "major": 39, "related_ge":  9, "related_elective":  0, "remaining_ge": 32, "remaining_elective": 40 },
    { "program_id": 16, "major": 30, "related_ge":  0, "related_elective":  0, "remaining_ge": 41, "remaining_elective": 49 },
    { "program_id": 17, "major": 33, "related_ge":  3, "related_elective":  0, "remaining_ge": 38, "remaining_elective": 46 },
    { "program_id": 18, "major": 33, "related_ge":  0, "related_elective":  0, "remaining_ge": 41, "remaining_elective": 46 },
    { "program_id": 19, "major": 33, "related_ge":  6, "related_elective":  0, "remaining_ge": 35, "remaining_elective": 46 },
    { "program_id": 20, "major": 33, "related_ge":  0, "related_elective":  0, "remaining_ge": 41, "remaining_elective": 46 },
    { "program_id": 21, "major": 33, "related_ge":  3, "related_elective":  0, "remaining_ge": 38, "remaining_elective": 46 },
    { "program_id": 22, "major": 36, "related_ge":  3, "related_elective":  0, "remaining_ge": 38, "remaining_elective": 43 },
    { "program_id": 23, "major": 33, "related_ge":  0, "related_elective":  0, "remaining_ge": 41, "remaining_elective": 46 },
    { "program_id": 24, "major": 36, "related_ge":  8, "related_elective":  6, "remaining_ge": 33, "remaining_elective": 37 },
    { "program_id": 25, "major": 33, "related_ge":  0, "related_elective":  0, "remaining_ge": 41, "remaining_elective": 46 },
    { "program_id": 26, "major": 33, "related_ge":  3, "related_elective":  0, "remaining_ge": 38, "remaining_elective": 46 },
    { "program_id": 27, "major": 33, "related_ge":  9, "related_elective":  0, "remaining_ge": 32, "remaining_elective": 46 },
    { "program_id": 28, "major": 36, "related_ge":  0, "related_elective":  0, "remaining_ge": 41, "remaining_elective": 43 },
    { "program_id": 29, "major": 30, "related_ge": 17, "related_elective":  4, "remaining_ge": 24, "remaining_elective": 45 },
    { "program_id": 30, "major": 30, "related_ge":  0, "related_elective":  0, "remaining_ge": 41, "remaining_elective": 49 },
    { "program_id": 31, "major": 33, "related_ge":  3, "related_elective":  0, "remaining_ge": 38, "remaining_elective": 46 },
    { "program_id": 32, "major": 30, "related_ge":  3, "related_elective":  0, "remaining_ge": 38, "remaining_elective": 49 },
    { "program_id": 33, "major": 30, "related_ge":  3, "related_elective":  0, "remaining_ge": 38, "remaining_elective": 49 },
    { "program_id": 34, "major": 33, "related_ge":  0, "related_elective":  0, "remaining_ge": 41, "remaining_elective": 46 },
    { "program_id": 35, "major": 30, "related_ge":  0, "related_elective":  0, "remaining_ge": 41, "remaining_elective": 49 }
]

drop_table('program_requirements',c)
c.execute('''
    CREATE TABLE program_requirements (
        id INTEGER PRIMARY KEY,
        program_id INTEGER,
        major INTEGER,
        related_ge INTEGER,
        related_elective INTEGER,
        remaining_ge INTEGER,
        remaining_elective INTEGER,
        total INTEGER DEFAULT 120,
        FOREIGN KEY(program_id) REFERENCES programs(id)
    )
''')

c.executemany('''
    INSERT INTO program_requirements (program_id, major, related_ge, related_elective, remaining_ge, remaining_elective)
        VALUES (:program_id, :major, :related_ge, :related_elective, :remaining_ge, :remaining_elective )
''', program_requirements)
#c.execute('UPDATE program_requirements SET total=120')
conn.commit()

In [10]:
################################################################
# "Program Requirement Courses" Table

program_requirement_courses = [
    { "program_id":  2, "course_type_id": 1, "course": "ACCT 220", "substitutions": "" },
    { "program_id":  2, "course_type_id": 1, "course": "ACCT 221", "substitutions": "" },
    { "program_id":  2, "course_type_id": 1, "course": "ACCT 310", "substitutions": "" },
    { "program_id":  2, "course_type_id": 1, "course": "ACCT 311", "substitutions": "" },
    { "program_id":  2, "course_type_id": 1, "course": "ACCT 321", "substitutions": "" },
    { "program_id":  2, "course_type_id": 1, "course": "ACCT 323", "substitutions": "" },
    { "program_id":  2, "course_type_id": 1, "course": "ACCT 326", "substitutions": "" },
    { "program_id":  2, "course_type_id": 1, "course": "ACCT 410", "substitutions": "ACCT 300+" },
    { "program_id":  2, "course_type_id": 1, "course": "ACCT 422", "substitutions": "" },
    { "program_id":  2, "course_type_id": 1, "course": "ACCT 424", "substitutions": "" },
    { "program_id":  2, "course_type_id": 1, "course": "ACCT 436", "substitutions": "ACCT 300+" },
    { "program_id":  2, "course_type_id": 1, "course": "ACCT 438", "substitutions": "ACCT 300+" },
    { "program_id":  2, "course_type_id": 6, "course": "ECON 201", "substitutions": "" },
    { "program_id":  2, "course_type_id": 6, "course": "ECON 203", "substitutions": "" },
    { "program_id":  2, "course_type_id": 6, "course": "STAT 200", "substitutions": "" },
    { "program_id":  2, "course_type_id": 6, "course": "WRTG 293", "substitutions": "COMM 390 | WRTG 394" },
    { "program_id":  2, "course_type_id": 7, "course": "BMGT 364", "substitutions": "" },
    { "program_id":  2, "course_type_id": 7, "course": "BMGT 380", "substitutions": "" },
    { "program_id":  2, "course_type_id": 7, "course": "FINC 330", "substitutions": "" },
    { "program_id":  2, "course_type_id": 7, "course": "MRKT 310", "substitutions": "" },
    { "program_id":  2, "course_type_id": 7, "course": "ACCT 411", "substitutions": "BMGT 496" },
    { "program_id":  5, "course_type_id": 1, "course": "BMGT 110", "substitutions": "" },
    { "program_id":  5, "course_type_id": 1, "course": "ACCT 220", "substitutions": "" },
    { "program_id":  5, "course_type_id": 1, "course": "ACCT 221", "substitutions": "" },
    { "program_id":  5, "course_type_id": 1, "course": "BMGT 364", "substitutions": "" },
    { "program_id":  5, "course_type_id": 1, "course": "BMGT 365", "substitutions": "" },
    { "program_id":  5, "course_type_id": 1, "course": "MRKT 310", "substitutions": "" },
    { "program_id":  5, "course_type_id": 1, "course": "BMGT 380", "substitutions": "" },
    { "program_id":  5, "course_type_id": 1, "course": "HRMN 300", "substitutions": "" },
    { "program_id":  5, "course_type_id": 1, "course": "FINC 330", "substitutions": "" },
    { "program_id":  5, "course_type_id": 1, "course": "BMGT 496", "substitutions": "" },
    { "program_id":  5, "course_type_id": 1, "course": "BMGT 495", "substitutions": "" },
    { "program_id":  5, "course_type_id": 6, "course": "ECON 201", "substitutions": "" },
    { "program_id":  5, "course_type_id": 6, "course": "ECON 203", "substitutions": "" },
    { "program_id":  5, "course_type_id": 6, "course": "IFSM 300", "substitutions": "" },
    { "program_id":  5, "course_type_id": 6, "course": "STAT 200", "substitutions": "" },
	{ "program_id":  6, "course_type_id": 1, "course": "SPCH 100", "substitutions": "SPCH 100+" },
	{ "program_id":  6, "course_type_id": 1, "course": "COMM 207", "substitutions": "COMM 100+" },
	{ "program_id":  6, "course_type_id": 1, "course": "JOUR 201", "substitutions": "" },
	{ "program_id":  6, "course_type_id": 1, "course": "COMM 300", "substitutions": "" },
	{ "program_id":  6, "course_type_id": 1, "course": "COMM 302", "substitutions": "" },
	{ "program_id":  6, "course_type_id": 1, "course": "SPCH 324", "substitutions": "" },
	{ "program_id":  6, "course_type_id": 1, "course": "JOUR 330", "substitutions": "JOUR 300+" },
	{ "program_id":  6, "course_type_id": 1, "course": "COMM 400", "substitutions": "COMM 300+" },
	{ "program_id":  6, "course_type_id": 1, "course": "SPCH 470", "substitutions": "SPCH 300+" },
	{ "program_id":  6, "course_type_id": 1, "course": "COMM 390", "substitutions": "COMM 300+" },
	{ "program_id":  6, "course_type_id": 1, "course": "COMM 495", "substitutions": "" },
	{ "program_id":  7, "course_type_id": 1, "course": "CMSC 115", "substitutions": "" },
	{ "program_id":  7, "course_type_id": 1, "course": "CMSC 215", "substitutions": "" },
	{ "program_id":  7, "course_type_id": 1, "course": "CMSC 310", "substitutions": "" },
	{ "program_id":  7, "course_type_id": 1, "course": "CMIT 265", "substitutions": "" },
	{ "program_id":  7, "course_type_id": 1, "course": "CMSC 315", "substitutions": "" },
	{ "program_id":  7, "course_type_id": 1, "course": "CMSC 320", "substitutions": "" },
	{ "program_id":  7, "course_type_id": 1, "course": "CMSC 330", "substitutions": "" },
	{ "program_id":  7, "course_type_id": 1, "course": "CMSC 335", "substitutions": "" },
	{ "program_id":  7, "course_type_id": 1, "course": "CMSC 345", "substitutions": "" },
	{ "program_id":  7, "course_type_id": 1, "course": "CMSC 430", "substitutions": "" },
	{ "program_id":  7, "course_type_id": 1, "course": "CMSC 451", "substitutions": "" },
	{ "program_id":  7, "course_type_id": 1, "course": "CMSC 495", "substitutions": "" },
	{ "program_id":  7, "course_type_id": 6, "course": "MATH 140", "substitutions": "" },
	{ "program_id":  7, "course_type_id": 6, "course": "CMSC 105", "substitutions": "" },
	{ "program_id":  7, "course_type_id": 7, "course": "MATH 141", "substitutions": "" },
	{ "program_id":  7, "course_type_id": 7, "course": "CMSC 150", "substitutions": "" },
	{ "program_id":  8, "course_type_id": 1, "course": "CCJS 100", "substitutions": "" },
	{ "program_id":  8, "course_type_id": 1, "course": "CCJS 230", "substitutions": "" },
	{ "program_id":  8, "course_type_id": 1, "course": "CCJS 340", "substitutions": "" },
	{ "program_id":  8, "course_type_id": 1, "course": "CCJS 345", "substitutions": "" },
	{ "program_id":  8, "course_type_id": 1, "course": "CCJS 350", "substitutions": "" },
	{ "program_id":  8, "course_type_id": 1, "course": "CCJS 360", "substitutions": "" },
	{ "program_id":  8, "course_type_id": 1, "course": "CCJS 380", "substitutions": "" },
	{ "program_id":  8, "course_type_id": 1, "course": "CCJS 341", "substitutions": "" },
	{ "program_id":  8, "course_type_id": 1, "course": "CCJS 352", "substitutions": "" },
	{ "program_id":  8, "course_type_id": 1, "course": "CCJS 497", "substitutions": "" },
	{ "program_id":  8, "course_type_id": 1, "course": "CCJS 495", "substitutions": "" },
	{ "program_id":  9, "course_type_id": 1, "course": "CSIA 300", "substitutions": "" },
	{ "program_id":  9, "course_type_id": 1, "course": "IFSM 304", "substitutions": "" },
	{ "program_id":  9, "course_type_id": 1, "course": "CMIT 265", "substitutions": "" },
	{ "program_id":  9, "course_type_id": 1, "course": "CMIT 320", "substitutions": "" },
	{ "program_id":  9, "course_type_id": 1, "course": "CSIA 310", "substitutions": "" },
	{ "program_id":  9, "course_type_id": 1, "course": "CSIA 350", "substitutions": "" },
	{ "program_id":  9, "course_type_id": 1, "course": "CSIA 360", "substitutions": "" },
	{ "program_id":  9, "course_type_id": 1, "course": "CSIA 413", "substitutions": "" },
	{ "program_id":  9, "course_type_id": 1, "course": "CSIA 459", "substitutions": "" },
	{ "program_id":  9, "course_type_id": 1, "course": "CMIT 425", "substitutions": "" },
	{ "program_id":  9, "course_type_id": 1, "course": "CSIA 485", "substitutions": "" },
	{ "program_id": 11, "course_type_id": 1, "course": "STAT 200", "substitutions": "" },
	{ "program_id": 11, "course_type_id": 1, "course": "DATA 230", "substitutions": "" },
	{ "program_id": 11, "course_type_id": 1, "course": "DATA 300", "substitutions": "" },
	{ "program_id": 11, "course_type_id": 1, "course": "CSIA 300", "substitutions": "" },
	{ "program_id": 11, "course_type_id": 1, "course": "DATA 320", "substitutions": "" },
	{ "program_id": 11, "course_type_id": 1, "course": "IFSM 330", "substitutions": "" },
	{ "program_id": 11, "course_type_id": 1, "course": "DATA 335", "substitutions": "" },
	{ "program_id": 11, "course_type_id": 1, "course": "DATA 430", "substitutions": "" },
	{ "program_id": 11, "course_type_id": 1, "course": "DATA 440", "substitutions": "" },
	{ "program_id": 11, "course_type_id": 1, "course": "DATA 445", "substitutions": "" },
	{ "program_id": 11, "course_type_id": 1, "course": "DATA 450", "substitutions": "" },
	{ "program_id": 11, "course_type_id": 1, "course": "DATA 460", "substitutions": "" },
	{ "program_id": 11, "course_type_id": 1, "course": "DATA 495", "substitutions": "" },
	{ "program_id": 11, "course_type_id": 6, "course": "DATA 200", "substitutions": "" },
	{ "program_id": 11, "course_type_id": 6, "course": "MATH 115", "substitutions": "" },
	{ "program_id": 14, "course_type_id": 1, "course": "ENHS 300", "substitutions": "" },
	{ "program_id": 14, "course_type_id": 1, "course": "ENHS 305", "substitutions": "" },
	{ "program_id": 14, "course_type_id": 1, "course": "ENHS 310", "substitutions": "" },
	{ "program_id": 14, "course_type_id": 1, "course": "ENHS 315", "substitutions": "" },
	{ "program_id": 14, "course_type_id": 1, "course": "ENHS 320", "substitutions": "" },
	{ "program_id": 14, "course_type_id": 1, "course": "ENHS 325", "substitutions": "" },
	{ "program_id": 14, "course_type_id": 1, "course": "ENHS 330", "substitutions": "" },
	{ "program_id": 14, "course_type_id": 1, "course": "ENHS 335", "substitutions": "" },
	{ "program_id": 14, "course_type_id": 1, "course": "ENHS 340", "substitutions": "" },
	{ "program_id": 14, "course_type_id": 1, "course": "ENHS 400", "substitutions": "" },
	{ "program_id": 14, "course_type_id": 1, "course": "ENHS 405", "substitutions": "" },
	{ "program_id": 14, "course_type_id": 1, "course": "ENHS 495", "substitutions": "" },
	{ "program_id": 14, "course_type_id": 6, "course": "CHEM 297", "substitutions": "" },
	{ "program_id": 14, "course_type_id": 6, "course": "MATH 115", "substitutions": "" },
	{ "program_id": 15, "course_type_id": 1, "course": "BMGT 364", "substitutions": "" },
	{ "program_id": 15, "course_type_id": 1, "course": "ACCT 220", "substitutions": "" },
	{ "program_id": 15, "course_type_id": 1, "course": "ACCT 221", "substitutions": "" },
	{ "program_id": 15, "course_type_id": 1, "course": "FINC 330", "substitutions": "" },
	{ "program_id": 15, "course_type_id": 1, "course": "FINC 335", "substitutions": "" },
	{ "program_id": 15, "course_type_id": 1, "course": "FINC 340", "substitutions": "" },
	{ "program_id": 15, "course_type_id": 1, "course": "FINC 351", "substitutions": "" },
	{ "program_id": 15, "course_type_id": 1, "course": "FINC 421", "substitutions": "" },
	{ "program_id": 15, "course_type_id": 1, "course": "FINC 430", "substitutions": "" },
	{ "program_id": 15, "course_type_id": 1, "course": "FINC 440", "substitutions": "" },
	{ "program_id": 15, "course_type_id": 1, "course": "FINC 460", "substitutions": "" },
	{ "program_id": 15, "course_type_id": 1, "course": "ECON 430", "substitutions": "" },
	{ "program_id": 15, "course_type_id": 1, "course": "FINC 495", "substitutions": "" },
	{ "program_id": 15, "course_type_id": 6, "course": "ECON 201", "substitutions": "" },
	{ "program_id": 15, "course_type_id": 6, "course": "ECON 203", "substitutions": "" },
	{ "program_id": 15, "course_type_id": 6, "course": "STAT 200", "substitutions": "" },
	{ "program_id": 17, "course_type_id": 1, "course": "GERO 100", "substitutions": "" },
	{ "program_id": 17, "course_type_id": 1, "course": "GERO 301", "substitutions": "" },
	{ "program_id": 17, "course_type_id": 1, "course": "GERO 302", "substitutions": "" },
	{ "program_id": 17, "course_type_id": 1, "course": "GERO 306", "substitutions": "" },
	{ "program_id": 17, "course_type_id": 1, "course": "GERO 311", "substitutions": "" },
	{ "program_id": 17, "course_type_id": 1, "course": "GERO 320", "substitutions": "" },
	{ "program_id": 17, "course_type_id": 1, "course": "GERO 338", "substitutions": "" },
	{ "program_id": 17, "course_type_id": 1, "course": "GERO 342", "substitutions": "" },
	{ "program_id": 17, "course_type_id": 1, "course": "GERO 390", "substitutions": "" },
	{ "program_id": 17, "course_type_id": 1, "course": "GERO 427", "substitutions": "" },
	{ "program_id": 17, "course_type_id": 1, "course": "GERO 486A", "substitutions": "" },
	{ "program_id": 17, "course_type_id": 6, "course": "STAT 200", "substitutions": "" },
	{ "program_id": 18, "course_type_id": 1, "course": "GRCO 100", "substitutions": "" },
	{ "program_id": 18, "course_type_id": 1, "course": "ARTT 110", "substitutions": "" },
	{ "program_id": 18, "course_type_id": 1, "course": "ARTT 120", "substitutions": "" },
	{ "program_id": 18, "course_type_id": 1, "course": "ARTT 210", "substitutions": "" },
	{ "program_id": 18, "course_type_id": 1, "course": "GRCO 230", "substitutions": "" },
	{ "program_id": 18, "course_type_id": 1, "course": "GRCO 350", "substitutions": "" },
	{ "program_id": 18, "course_type_id": 1, "course": "GRCO 354", "substitutions": "" },
	{ "program_id": 18, "course_type_id": 1, "course": "GRCO 355", "substitutions": "" },
	{ "program_id": 18, "course_type_id": 1, "course": "GRCO 450", "substitutions": "" },
	{ "program_id": 18, "course_type_id": 1, "course": "GRCO 479", "substitutions": "" },
	{ "program_id": 18, "course_type_id": 1, "course": "GRCO 495", "substitutions": "" },
	{ "program_id": 19, "course_type_id": 1, "course": "HMGT 300", "substitutions": "" },
	{ "program_id": 19, "course_type_id": 1, "course": "HMGT 307", "substitutions": "" },
	{ "program_id": 19, "course_type_id": 1, "course": "HMGT 310", "substitutions": "" },
	{ "program_id": 19, "course_type_id": 1, "course": "HMGT 320", "substitutions": "" },
	{ "program_id": 19, "course_type_id": 1, "course": "HMGT 322", "substitutions": "" },
	{ "program_id": 19, "course_type_id": 1, "course": "HMGT 335", "substitutions": "" },
	{ "program_id": 19, "course_type_id": 1, "course": "HMGT 372", "substitutions": "" },
	{ "program_id": 19, "course_type_id": 1, "course": "HMGT 400", "substitutions": "" },
	{ "program_id": 19, "course_type_id": 1, "course": "HMGT 420", "substitutions": "" },
	{ "program_id": 19, "course_type_id": 1, "course": "HMGT 435", "substitutions": "" },
	{ "program_id": 19, "course_type_id": 1, "course": "HMGT 495", "substitutions": "" },
	{ "program_id": 19, "course_type_id": 6, "course": "IFSM 305", "substitutions": "" },
	{ "program_id": 19, "course_type_id": 6, "course": "STAT 200", "substitutions": "" },
	{ "program_id": 21, "course_type_id": 1, "course": "HMLS 302", "substitutions": "" },
	{ "program_id": 21, "course_type_id": 1, "course": "HMLS 310", "substitutions": "" },
	{ "program_id": 21, "course_type_id": 1, "course": "HMLS 406", "substitutions": "" },
	{ "program_id": 21, "course_type_id": 1, "course": "HMLS 408", "substitutions": "" },
	{ "program_id": 21, "course_type_id": 1, "course": "HMLS 414", "substitutions": "" },
	{ "program_id": 21, "course_type_id": 1, "course": "HMLS 416", "substitutions": "" },
	{ "program_id": 21, "course_type_id": 1, "course": "PSAD 410", "substitutions": "" },
	{ "program_id": 21, "course_type_id": 1, "course": "PSAD 414", "substitutions": "" },
	{ "program_id": 21, "course_type_id": 1, "course": "PSAD 416", "substitutions": "" },
	{ "program_id": 21, "course_type_id": 1, "course": "HMLS 304", "substitutions": "" },
	{ "program_id": 21, "course_type_id": 1, "course": "HMLS 495", "substitutions": "" },
	{ "program_id": 21, "course_type_id": 6, "course": "IFSM 300", "substitutions": "" },
    { "program_id": 22, "course_type_id": 1, "course": "HRMN 300", "substitutions": "" },
    { "program_id": 22, "course_type_id": 1, "course": "HRMN 302", "substitutions": "" },
    { "program_id": 22, "course_type_id": 1, "course": "HRMN 362", "substitutions": "" },
    { "program_id": 22, "course_type_id": 1, "course": "HRMN 367", "substitutions": "" },
    { "program_id": 22, "course_type_id": 1, "course": "HRMN 395", "substitutions": "" },
    { "program_id": 22, "course_type_id": 1, "course": "HRMN 400", "substitutions": "" },
    { "program_id": 22, "course_type_id": 1, "course": "HRMN 406", "substitutions": "HRMN 410" },
    { "program_id": 22, "course_type_id": 1, "course": "BMGT 364", "substitutions": "" },
    { "program_id": 22, "course_type_id": 1, "course": "FINC 331", "substitutions": "" },
    { "program_id": 22, "course_type_id": 1, "course": "HRMN 408", "substitutions": "" },
    { "program_id": 22, "course_type_id": 1, "course": "HRMN 467", "substitutions": "" },
    { "program_id": 22, "course_type_id": 1, "course": "HRMN 495", "substitutions": "" },
    { "program_id": 22, "course_type_id": 6, "course": "IFSM 300", "substitutions": "" },
    { "program_id": 23, "course_type_id": 1, "course": "HUMN 100", "substitutions": "" },
    { "program_id": 23, "course_type_id": 1, "course": "PHIL 100", "substitutions": "" },
    { "program_id": 23, "course_type_id": 1, "course": "PHIL 140", "substitutions": "" },
    { "program_id": 23, "course_type_id": 1, "course": "HIST 115", "substitutions": "HIST 116 | HIST 141 | HIST 142" },
    { "program_id": 23, "course_type_id": 1, "course": "MUSC 210", "substitutions": "MUSC 100+" },
    { "program_id": 23, "course_type_id": 1, "course": "ARTH 372", "substitutions": "ARTH 300+" },
    { "program_id": 23, "course_type_id": 1, "course": "PHIL 304", "substitutions": "PHIL 300+" },
    { "program_id": 23, "course_type_id": 1, "course": "HUMN 351", "substitutions": "HUMN 300+" },
    { "program_id": 23, "course_type_id": 1, "course": "PHIL 349", "substitutions": "PHIL 300+" },
    { "program_id": 23, "course_type_id": 1, "course": "ENGL 406", "substitutions": "ENGL 300+" },
    { "program_id": 23, "course_type_id": 1, "course": "HUMN 495", "substitutions": "" },
    { "program_id": 25, "course_type_id": 1, "course": "LGST 101", "substitutions": "" },
    { "program_id": 25, "course_type_id": 1, "course": "LGST 200", "substitutions": "" },
    { "program_id": 25, "course_type_id": 1, "course": "LGST 201", "substitutions": "" },
    { "program_id": 25, "course_type_id": 1, "course": "LGST 204", "substitutions": "" },
    { "program_id": 25, "course_type_id": 1, "course": "LGST 301", "substitutions": "" },
    { "program_id": 25, "course_type_id": 1, "course": "LGST 312", "substitutions": "" },
    { "program_id": 25, "course_type_id": 1, "course": "LGST 315", "substitutions": "" },
    { "program_id": 25, "course_type_id": 1, "course": "LGST 320", "substitutions": "" },
    { "program_id": 25, "course_type_id": 1, "course": "LGST 325", "substitutions": "" },
    { "program_id": 25, "course_type_id": 1, "course": "LGST 340", "substitutions": "" },
    { "program_id": 25, "course_type_id": 1, "course": "LGST 495", "substitutions": "" },
    { "program_id": 26, "course_type_id": 1, "course": "IFSM 300", "substitutions": "" }, 
    { "program_id": 26, "course_type_id": 1, "course": "IFSM 301", "substitutions": "" }, 
    { "program_id": 26, "course_type_id": 1, "course": "IFSM 304", "substitutions": "" }, 
    { "program_id": 26, "course_type_id": 1, "course": "CSIA 300", "substitutions": "" }, 
    { "program_id": 26, "course_type_id": 1, "course": "IFSM 310", "substitutions": "" }, 
    { "program_id": 26, "course_type_id": 1, "course": "IFSM 311", "substitutions": "" }, 
    { "program_id": 26, "course_type_id": 1, "course": "IFSM 330", "substitutions": "" }, 
    { "program_id": 26, "course_type_id": 1, "course": "IFSM 370", "substitutions": "" }, 
    { "program_id": 26, "course_type_id": 1, "course": "IFSM 438", "substitutions": "" }, 
    { "program_id": 26, "course_type_id": 1, "course": "IFSM 461", "substitutions": "" }, 
    { "program_id": 26, "course_type_id": 1, "course": "IFSM 495", "substitutions": "" }, 
    { "program_id": 26, "course_type_id": 6, "course": "CMSC 105", "substitutions": "" }, 
    { "program_id": 27, "course_type_id": 1, "course": "BMGT 160", "substitutions": "" },
    { "program_id": 27, "course_type_id": 1, "course": "ACCT 301", "substitutions": "ACCT 220" },
    { "program_id": 27, "course_type_id": 1, "course": "BMGT 364", "substitutions": "" },
    { "program_id": 27, "course_type_id": 1, "course": "BMGT 365", "substitutions": "ACCT 300+ | BMGT 300+ | FINC 300+ | HRMN 300+ | MRKT 300+" },
    { "program_id": 27, "course_type_id": 1, "course": "BMGT 464", "substitutions": "BMGT 465" },
    { "program_id": 27, "course_type_id": 1, "course": "BMGT 317", "substitutions": "" },
    { "program_id": 27, "course_type_id": 1, "course": "BMGT 305", "substitutions": "" },
    { "program_id": 27, "course_type_id": 1, "course": "BMGT 304", "substitutions": "ACCT 300+ | BMGT 300+ | FINC 300+ | HRMN 300+ | MRKT 300+" },
    { "program_id": 27, "course_type_id": 1, "course": "BMGT 484", "substitutions": "" },
    { "program_id": 27, "course_type_id": 1, "course": "BMGT 496", "substitutions": "" },
    { "program_id": 27, "course_type_id": 1, "course": "BMGT 485", "substitutions": "" },
    { "program_id": 27, "course_type_id": 6, "course": "ECON 201", "substitutions": "" },
    { "program_id": 27, "course_type_id": 6, "course": "IFSM 300", "substitutions": "" },
    { "program_id": 27, "course_type_id": 6, "course": "STAT 200", "substitutions": "" },
    { "program_id": 28, "course_type_id": 1, "course": "BMGT 110", "substitutions": "" },
    { "program_id": 28, "course_type_id": 1, "course": "BMGT 330", "substitutions": "" },
    { "program_id": 28, "course_type_id": 1, "course": "MRKT 310", "substitutions": "" },
    { "program_id": 28, "course_type_id": 1, "course": "MRKT 354", "substitutions": "" },
    { "program_id": 28, "course_type_id": 1, "course": "MRKT 394", "substitutions": "" },
    { "program_id": 28, "course_type_id": 1, "course": "MRKT 410", "substitutions": "" },
    { "program_id": 28, "course_type_id": 1, "course": "MRKT 412", "substitutions": "" },
    { "program_id": 28, "course_type_id": 1, "course": "MRKT 458", "substitutions": "" },
    { "program_id": 28, "course_type_id": 1, "course": "MRKT 311", "substitutions": "" },
    { "program_id": 28, "course_type_id": 1, "course": "MRKT 314", "substitutions": "" },
    { "program_id": 28, "course_type_id": 1, "course": "MRKT 454", "substitutions": "" },
    { "program_id": 28, "course_type_id": 1, "course": "MRKT 495", "substitutions": "" },
    { "program_id": 30, "course_type_id": 1, "course": "GVPT 100", "substitutions": "" },
    { "program_id": 30, "course_type_id": 1, "course": "GVPT 101", "substitutions": "" },
    { "program_id": 30, "course_type_id": 1, "course": "GVPT 170", "substitutions": "GVPT 200" },
    { "program_id": 30, "course_type_id": 1, "course": "GVPT 210", "substitutions": "" },
    { "program_id": 30, "course_type_id": 1, "course": "GVPT 280", "substitutions": "" },
    { "program_id": 30, "course_type_id": 1, "course": "GVPT 306", "substitutions": "" },
    { "program_id": 30, "course_type_id": 1, "course": "GVPT 403", "substitutions": "GVPT 300+" },
    { "program_id": 30, "course_type_id": 1, "course": "GVPT 406", "substitutions": "" },
    { "program_id": 30, "course_type_id": 1, "course": "GVPT 457", "substitutions": "GVPT 300+" },
    { "program_id": 30, "course_type_id": 1, "course": "GVPT 495", "substitutions": "" },
    { "program_id": 31, "course_type_id": 1, "course": "PSYC 100", "substitutions": "" }, 
    { "program_id": 31, "course_type_id": 1, "course": "PSYC 220", "substitutions": "" }, 
    { "program_id": 31, "course_type_id": 1, "course": "PSYC 251", "substitutions": "" }, 
    { "program_id": 31, "course_type_id": 1, "course": "PSYC 300", "substitutions": "" }, 
    { "program_id": 31, "course_type_id": 1, "course": "PSYC 301", "substitutions": "" }, 
    { "program_id": 31, "course_type_id": 1, "course": "PSYC 310", "substitutions": "" }, 
    { "program_id": 31, "course_type_id": 1, "course": "PSYC 335", "substitutions": "" }, 
    { "program_id": 31, "course_type_id": 1, "course": "PSYC 341", "substitutions": "" }, 
    { "program_id": 31, "course_type_id": 1, "course": "PSYC 353", "substitutions": "" }, 
    { "program_id": 31, "course_type_id": 1, "course": "PSYC 436", "substitutions": "" }, 
    { "program_id": 31, "course_type_id": 1, "course": "PSYC 495", "substitutions": "" }, 
    { "program_id": 31, "course_type_id": 6, "course": "STAT 200", "substitutions": "" },
    { "program_id": 32, "course_type_id": 1, "course": "PSAD 302", "substitutions": "" }, 
    { "program_id": 32, "course_type_id": 1, "course": "PSAD 304", "substitutions": "" }, 
    { "program_id": 32, "course_type_id": 1, "course": "PSAD 306", "substitutions": "" }, 
    { "program_id": 32, "course_type_id": 1, "course": "PSAD 408", "substitutions": "" }, 
    { "program_id": 32, "course_type_id": 1, "course": "PSAD 410", "substitutions": "" }, 
    { "program_id": 32, "course_type_id": 1, "course": "PSAD 414", "substitutions": "" }, 
    { "program_id": 32, "course_type_id": 1, "course": "PSAD 416", "substitutions": "" }, 
    { "program_id": 32, "course_type_id": 1, "course": "FINC 331", "substitutions": "" }, 
    { "program_id": 32, "course_type_id": 1, "course": "BMGT 317", "substitutions": "" }, 
    { "program_id": 32, "course_type_id": 1, "course": "PSAD 495", "substitutions": "" }, 
    { "program_id": 32, "course_type_id": 6, "course": "IFSM 300", "substitutions": "" },
    { "program_id": 34, "course_type_id": 1, "course": "CMSC 115", "substitutions": "" },
    { "program_id": 34, "course_type_id": 1, "course": "CMSC 215", "substitutions": "" },
    { "program_id": 34, "course_type_id": 1, "course": "CMSC 320", "substitutions": "" },
    { "program_id": 34, "course_type_id": 1, "course": "SDEV 300", "substitutions": "" },
    { "program_id": 34, "course_type_id": 1, "course": "SDEV 325", "substitutions": "" },
    { "program_id": 34, "course_type_id": 1, "course": "SDEV 350", "substitutions": "" },
    { "program_id": 34, "course_type_id": 1, "course": "SDEV 360", "substitutions": "" },
    { "program_id": 34, "course_type_id": 1, "course": "SDEV 400", "substitutions": "" },
    { "program_id": 34, "course_type_id": 1, "course": "SDEV 425", "substitutions": "" },
    { "program_id": 34, "course_type_id": 1, "course": "SDEV 460", "substitutions": "" },
    { "program_id": 34, "course_type_id": 1, "course": "CMSC 495", "substitutions": "" }
]

drop_table('program_requirement_courses',c)
c.execute('''
    CREATE TABLE program_requirement_courses (
        id INTEGER PRIMARY KEY,
        program_id INTEGER,
        course_type_id INTEGER,
        course TEXT,
        substitutions TEXT,
        FOREIGN KEY(program_id) REFERENCES programs(id),
        FOREIGN KEY(course_type_id) REFERENCES course_type(id)
    )
''')

c.executemany('''
    INSERT INTO program_requirement_courses (program_id, course_type_id, course, substitutions) 
       VALUES (:program_id, :course_type_id, :course, :substitutions)
''', program_requirement_courses)
conn.commit()

In [9]:
################################################################
# "Program Sequence" Table
# For now, this is the program sequence recommended by the catalog

program_sequence = [
    { "seq":  1, "program_id": 5, "name": "PACE 111B", "course_type_id": 3, "source": "catalog" },
    { "seq":  2, "program_id": 5, "name": "LIBS 150", "course_type_id": 3, "source": "catalog" },
    { "seq":  3, "program_id": 5, "name": "WRTG 111", "course_type_id": 3, "source": "catalog" },
    { "seq":  4, "program_id": 5, "name": "WRTG 112", "course_type_id": 3, "source": "catalog" },
    { "seq":  5, "program_id": 5, "name": "NUTR 100", "course_type_id": 3, "source": "catalog" },
    { "seq":  6, "program_id": 5, "name": "BMGT 110", "course_type_id": 1, "source": "catalog" },
    { "seq":  7, "program_id": 5, "name": "SPCH 100", "course_type_id": 3, "source": "catalog" },
    { "seq":  8, "program_id": 5, "name": "STAT 200", "course_type_id": 2, "source": "catalog" },
    { "seq":  9, "program_id": 5, "name": "IFSM 300", "course_type_id": 2, "source": "catalog" },
    { "seq": 10, "program_id": 5, "name": "ACCT 220", "course_type_id": 1, "source": "catalog" },
    { "seq": 11, "program_id": 5, "name": "HUMN 100", "course_type_id": 3, "source": "catalog" },
    { "seq": 12, "program_id": 5, "name": "BIOL 103", "course_type_id": 3, "source": "catalog" },
    { "seq": 13, "program_id": 5, "name": "ECON 201", "course_type_id": 2, "source": "catalog" },
    { "seq": 14, "program_id": 5, "name": "ARTH 334", "course_type_id": 3, "source": "catalog" },
    { "seq": 15, "program_id": 5, "name": "ELECTIVE", "course_type_id": 4, "source": "catalog" },
    { "seq": 16, "program_id": 5, "name": "ECON 203", "course_type_id": 2, "source": "catalog" },
    { "seq": 17, "program_id": 5, "name": "ACCT 221", "course_type_id": 1, "source": "catalog" },
    { "seq": 18, "program_id": 5, "name": "ELECTIVE", "course_type_id": 4, "source": "catalog" },
    { "seq": 19, "program_id": 5, "name": "BMGT 364", "course_type_id": 1, "source": "catalog" },
    { "seq": 20, "program_id": 5, "name": "ELECTIVE", "course_type_id": 4, "source": "catalog" },
    { "seq": 21, "program_id": 5, "name": "BMGT 365", "course_type_id": 1, "source": "catalog" },
    { "seq": 22, "program_id": 5, "name": "ELECTIVE", "course_type_id": 4, "source": "catalog" },
    { "seq": 23, "program_id": 5, "name": "MRKT 310", "course_type_id": 1, "source": "catalog" },
    { "seq": 24, "program_id": 5, "name": "WRTG 394", "course_type_id": 3, "source": "catalog" },
    { "seq": 25, "program_id": 5, "name": "ELECTIVE", "course_type_id": 4, "source": "catalog" },
    { "seq": 26, "program_id": 5, "name": "BMGT 380", "course_type_id": 1, "source": "catalog" },
    { "seq": 27, "program_id": 5, "name": "ELECTIVE", "course_type_id": 4, "source": "catalog" },
    { "seq": 28, "program_id": 5, "name": "ELECTIVE", "course_type_id": 4, "source": "catalog" },
    { "seq": 29, "program_id": 5, "name": "HRMN 300", "course_type_id": 1, "source": "catalog" },
    { "seq": 30, "program_id": 5, "name": "ELECTIVE", "course_type_id": 4, "source": "catalog" },
    { "seq": 31, "program_id": 5, "name": "ELECTIVE", "course_type_id": 4, "source": "catalog" },
    { "seq": 32, "program_id": 5, "name": "FINC 330", "course_type_id": 1, "source": "catalog" },
    { "seq": 33, "program_id": 5, "name": "ELECTIVE", "course_type_id": 4, "source": "catalog" },
    { "seq": 34, "program_id": 5, "name": "ELECTIVE", "course_type_id": 4, "source": "catalog" },
    { "seq": 35, "program_id": 5, "name": "BMGT 496", "course_type_id": 1, "source": "catalog" },
    { "seq": 36, "program_id": 5, "name": "ELECTIVE", "course_type_id": 4, "source": "catalog" },
    { "seq": 37, "program_id": 5, "name": "ELECTIVE", "course_type_id": 4, "source": "catalog" },
    { "seq": 38, "program_id": 5, "name": "ELECTIVE", "course_type_id": 4, "source": "catalog" },
    { "seq": 39, "program_id": 5, "name": "ELECTIVE", "course_type_id": 4, "source": "catalog" },
    { "seq": 40, "program_id": 5, "name": "BMGT 495", "course_type_id": 1, "source": "catalog" },
    { "seq": 41, "program_id": 5, "name": "CAPSTONE", "course_type_id": 4, "source": "catalog" }
]

drop_table('program_sequence',c)
c.execute('''
    CREATE TABLE program_sequence (
        seq INTEGER,
        name TEXT,
        program_id INTEGER,
        class_id INTEGER DEFAULT 0,
        course_type_id INTEGER,
        source TEXT,
        FOREIGN KEY(program_id) REFERENCES programs(id),
        FOREIGN KEY(class_id) REFERENCES classes(id),          
        FOREIGN KEY(course_type_id) REFERENCES course_type(id)
    )
''')
c.executemany('''
    INSERT INTO program_sequence (seq, name, program_id, course_type_id, source) 
        VALUES (:seq, :name, :program_id, :course_type_id, :source)
    '''
    , program_sequence
)

c.execute('''
    UPDATE program_sequence
        SET class_id = (
            SELECT id
            FROM classes
            WHERE classes.name = program_sequence.name
        )
''')
conn.commit()

In [10]:
################################################################
# Roles Table
# These are the roles for someone logging into the app
#   student:   can select a major and create a schedule
#   counselor: can select a student and do everything for them
#   admin: for now, the same as counselor with potentially additional admin tasks

roles = [
    { "id": 1, "type": "student"  },
    { "id": 2, "type": "counselor" },
    { "id": 3, "type": "admin" }
]

drop_table('roles',c)
c.execute('''
    CREATE TABLE roles (
        id INTEGER PRIMARY KEY,
        type TEXT
    )
''')

c.executemany('INSERT INTO roles VALUES (:id, :type)', roles)
conn.commit()

In [11]:
################################################################
# Resident status Table

resident_status = [
    { "id": 1, "type": "in-state" },
    { "id": 2, "type": "out-of-state" },
    { "id": 3, "type": "military" }
]

drop_table('resident_status',c)
c.execute('''
    CREATE TABLE resident_status (
        id INTEGER PRIMARY KEY,
        type TEXT
    )
''')

c.executemany('INSERT INTO resident_status VALUES (:id, :type)', resident_status)
conn.commit()

In [12]:
################################################################
# Tuition Table

tuition = [
    { "term": "Winter 2024", "program": "undergraduate",       "resident_status_id": 1, "cost":  318 },
    { "term": "Winter 2024", "program": "graduate",            "resident_status_id": 1, "cost":  524 },
    { "term": "Winter 2024", "program": "specialty_graduate",  "resident_status_id": 1, "cost":  694 },
    { "term": "Winter 2024", "program": "doctoral",            "resident_status_id": 1, "cost": 1087 },
    { "term": "Spring 2024", "program": "undergraduate",       "resident_status_id": 1, "cost":  318 },
    { "term": "Spring 2024", "program": "graduate",            "resident_status_id": 1, "cost":  524 },
    { "term": "Spring 2024", "program": "specialty_graduate",  "resident_status_id": 1, "cost":  694 },
    { "term": "Spring 2024", "program": "doctoral",            "resident_status_id": 1, "cost": 1087 },
    { "term": "Summer 2024", "program": "undergraduate",       "resident_status_id": 1, "cost":  324 },
    { "term": "Summer 2024", "program": "graduate",            "resident_status_id": 1, "cost":  534 },
    { "term": "Summer 2024", "program": "specialty_graduate",  "resident_status_id": 1, "cost":  694 },
    { "term": "Summer 2024", "program": "doctoral",            "resident_status_id": 1, "cost": 1087 },
    { "term": "Winter 2024", "program": "undergraduate",       "resident_status_id": 2, "cost":  499 },
    { "term": "Winter 2024", "program": "graduate",            "resident_status_id": 2, "cost":  659 },
    { "term": "Winter 2024", "program": "specialty_graduate",  "resident_status_id": 2, "cost":  694 },
    { "term": "Winter 2024", "program": "doctoral",            "resident_status_id": 2, "cost": 1087 },
    { "term": "Spring 2024", "program": "undergraduate",       "resident_status_id": 2, "cost":  499 },
    { "term": "Spring 2024", "program": "graduate",            "resident_status_id": 2, "cost":  659 },
    { "term": "Spring 2024", "program": "specialty_graduate",  "resident_status_id": 2, "cost":  694 },
    { "term": "Spring 2024", "program": "doctoral",            "resident_status_id": 2, "cost": 1087 },
    { "term": "Summer 2024", "program": "undergraduate",       "resident_status_id": 2, "cost":  499 },
    { "term": "Summer 2024", "program": "graduate",            "resident_status_id": 2, "cost":  659 },
    { "term": "Summer 2024", "program": "specialty_graduate",  "resident_status_id": 2, "cost":  694 },
    { "term": "Summer 2024", "program": "doctoral",            "resident_status_id": 2, "cost": 1087 },
    { "term": "Winter 2024", "program": "undergraduate",       "resident_status_id": 3, "cost":  250 },
    { "term": "Winter 2024", "program": "graduate",            "resident_status_id": 3, "cost":  480 },
    { "term": "Winter 2024", "program": "specialty_graduate",  "resident_status_id": 3, "cost":  480 },
    { "term": "Winter 2024", "program": "doctoral",            "resident_status_id": 3, "cost": 1087 },
    { "term": "Spring 2024", "program": "undergraduate",       "resident_status_id": 3, "cost":  250 },
    { "term": "Spring 2024", "program": "graduate",            "resident_status_id": 3, "cost":  480 },
    { "term": "Spring 2024", "program": "specialty_graduate",  "resident_status_id": 3, "cost":  480 },
    { "term": "Spring 2024", "program": "doctoral",            "resident_status_id": 3, "cost": 1087 },
    { "term": "Summer 2024", "program": "undergraduate",       "resident_status_id": 3, "cost":  250 },
    { "term": "Summer 2024", "program": "graduate",            "resident_status_id": 3, "cost":  336 },
    { "term": "Summer 2024", "program": "specialty_graduate",  "resident_status_id": 3, "cost":  336 },
    { "term": "Summer 2024", "program": "doctoral",            "resident_status_id": 3, "cost": 1087 }
]

drop_table('tuition',c)
c.execute('''
    CREATE TABLE tuition (
        id INTEGER PRIMARY KEY,
        term TEXT,
        program TEXT,
        resident_status_id INTEGER,
        cost INTEGER,
        FOREIGN KEY(resident_status_id) REFERENCES resident_status(id)
    )
''')

c.executemany('INSERT INTO tuition(term, program, resident_status_id, cost) VALUES (:term, :program, :resident_status_id, :cost)', tuition )
conn.commit()

In [13]:
################################################################
# Users Table

users = [
    { 
        "id": 1, 
        "role_id": 1, 
        "username": "johndoe", 
        "firstname": "John", 
        "lastname": "Doe"
    },
    { 
        "id": 2, 
        "role_id": 1, 
        "username": "janedoe", 
        "firstname": "Jane", 
        "lastname": "Doe" 
    },
    { 
        "id": 3, 
        "role_id": 1, 
        "username": "jimdoe", 
        "firstname": "Jim", 
        "lastname": "Doe" 
    },
    { 
        "id": 4, 
        "role_id": 1, 
        "username": "major", 
        "firstname": "Sergeant", 
        "lastname": "Major" 
    },
    { 
        "id": 5, 
        "role_id": 2, 
        "username": "counselor", 
        "firstname": "Dr.", 
        "lastname": "Counselor" 
    },
    { 
        "id": 6, 
        "role_id": 3, 
        "username": "admin", 
        "firstname": "The", 
        "lastname": "Administrator" 
    },
]

drop_table('users',c)
c.execute('''
    CREATE TABLE users (
        id INTEGER PRIMARY KEY,
        role_id INTEGER,
        username TEXT,
        firstname TEXT,
        lastname TEXT,
        FOREIGN KEY(role_id) REFERENCES roles(id)
    )
''')

c.executemany('INSERT INTO users VALUES (:id, :role_id, :username, :firstname, :lastname)', users)
conn.commit()

In [14]:
################################################################
## "Student Info" Table
# This table will contain information on students, including
#   - what stage they are in the app
#     stage = ['new', 'profiled', 'program_chosen', 'schedule_created']
#   - if a major is chosen (program_id)
#     program_id = 0 means major has not yet been chosen
#   - transfer_credits (0 or 1)
#   - financial_aid (0 or 1)
#   - if the student is in-state, out-of-state, or military
#   - profile: full-time, part-time, evening, unknown
# 

student_info = [
    { 
        "id": 1, 
        "user_id": 1, 
        "resident_status_id": 1,
        "transfer_credits": 0,
        "financial_aid": 0,
        "stage": "schedule_created",
        "program_id": 10, 
        "profile": "full-time",
        "notes": "John Doe new student program selected"
    },
    { 
        "id": 2, 
        "user_id": 3, 
        "resident_status_id": 1 , 
        "transfer_credits": 0,
        "financial_aid": 0,
        "stage": "new",
        "program_id": 0, 
        "profile": "unknown",
        "notes": "Jim Doe new student no program selected"
    },
    { 
        "id": 3, 
        "user_id": 2, 
        "resident_status_id": 2, 
        "transfer_credits": 0,
        "financial_aid": 0,
        "stage": "program_chosen",
        "program_id": 10, 
        "profile": "part-time",
        "notes": "Jane Doe with transfer credits" 
    },
    { 
        "id": 4, 
        "user_id": 4, 
        "resident_status_id": 3 , 
        "transfer_credits": 0,
        "financial_aid": 0,
        "stage": "new",
        "program_id": 0, 
        "profile": "evening",
        "notes": "Sergeant Major - evening school" 
    } 
]

drop_table('student_info',c)
c.execute('''
    CREATE TABLE student_info (
        id INTEGER PRIMARY KEY,
        user_id INTEGER,
        resident_status_id INTEGER,
        transfer_credits INTEGER,
        financial_aid INTEGER,
        stage TEXT,
        program_id INTEGER,
        profile TEXT,
        notes TEXT,
        FOREIGN KEY(user_id) REFERENCES users(id),
        FOREIGN KEY(resident_status_id) REFERENCES resident_status(id)
        FOREIGN KEY(program_id) REFERENCES programs(id)
    )
''')

c.executemany('''
    INSERT INTO student_info 
        VALUES (:id, :user_id, :resident_status_id, :transfer_credits, :financial_aid, :stage, :program_id, :profile, :notes)
    ''', student_info)
conn.commit()

In [17]:
################################################################
# "Student Progress" Table
# This is the table that will store student programs and progress for different students
# This is the database table that the D3 script pulls from currently
# Will need to rebuild and overwrite as optimization, etc. occurs

student_progress = [
    { "student_info_id": 10, "seq":  1, "name": "PACE 111B", "credits": 3, "type": "general",  "completed": 0, "period":  1, "session": 1, "prerequisite": ""                                },
    { "student_info_id": 10, "seq":  2, "name": "LIBS 150",  "credits": 1, "type": "general",  "completed": 0, "period":  1, "session": 1, "prerequisite": ""                                },
    { "student_info_id": 10, "seq":  3, "name": "WRTG 111",  "credits": 3, "type": "general",  "completed": 0, "period":  1, "session": 3, "prerequisite": ""                                },
    { "student_info_id": 10, "seq":  4, "name": "WRTG 112",  "credits": 3, "type": "general",  "completed": 0, "period":  2, "session": 1, "prerequisite": ""                                },
    { "student_info_id": 10, "seq":  5, "name": "NUTR 100",  "credits": 3, "type": "general",  "completed": 0, "period":  1, "session": 3, "prerequisite": ""                                },
    { "student_info_id": 10, "seq":  6, "name": "BMGT 110",  "credits": 3, "type": "major",    "completed": 0, "period":  2, "session": 2, "prerequisite": ""                                },
    { "student_info_id": 10, "seq":  7, "name": "SPCH 100",  "credits": 3, "type": "general",  "completed": 0, "period":  3, "session": 1, "prerequisite": ""                                },
    { "student_info_id": 10, "seq":  8, "name": "STAT 200",  "credits": 3, "type": "required", "completed": 0, "period":  3, "session": 2, "prerequisite": ""                                },
    { "student_info_id": 10, "seq":  9, "name": "IFSM 300",  "credits": 3, "type": "required", "completed": 0, "period":  4, "session": 1, "prerequisite": ""                                },
    { "student_info_id": 10, "seq": 10, "name": "ACCT 220",  "credits": 3, "type": "major",    "completed": 0, "period":  4, "session": 2, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 11, "name": "HUMN 100",  "credits": 3, "type": "general",  "completed": 0, "period":  4, "session": 3, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 12, "name": "BIOL 103",  "credits": 4, "type": "general",  "completed": 0, "period":  5, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 13, "name": "ECON 201",  "credits": 3, "type": "required", "completed": 0, "period":  5, "session": 3, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 14, "name": "ARTH 334",  "credits": 3, "type": "general",  "completed": 0, "period":  5, "session": 2, "prerequisite": ""                                },
    { "student_info_id": 10, "seq": 15, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period":  6, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 16, "name": "ECON 203",  "credits": 3, "type": "required", "completed": 0, "period":  6, "session": 2, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 17, "name": "ACCT 221",  "credits": 3, "type": "major",    "completed": 0, "period":  7, "session": 1, "prerequisite": "ACCT 220"                        }, 
    { "student_info_id": 10, "seq": 18, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period":  7, "session": 2, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 19, "name": "BMGT 364",  "credits": 3, "type": "major",    "completed": 0, "period":  7, "session": 3, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 20, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period":  8, "session": 2, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 21, "name": "BMGT 365",  "credits": 3, "type": "major",    "completed": 0, "period":  8, "session": 1, "prerequisite": "BMGT 364"                        }, 
    { "student_info_id": 10, "seq": 22, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period":  9, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 23, "name": "MRKT 310",  "credits": 3, "type": "major",    "completed": 0, "period":  8, "session": 3, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 24, "name": "WRTG 394",  "credits": 3, "type": "general",  "completed": 0, "period":  9, "session": 2, "prerequisite": "WRTG 112"                        }, 
    { "student_info_id": 10, "seq": 25, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period":  9, "session": 3, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 26, "name": "BMGT 380",  "credits": 3, "type": "major",    "completed": 0, "period": 10, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 27, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period": 10, "session": 2, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 28, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period": 11, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 29, "name": "HRMN 300",  "credits": 3, "type": "major",    "completed": 0, "period": 11, "session": 2, "prerequisite": ""                                },
    { "student_info_id": 10, "seq": 30, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period": 11, "session": 3, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 31, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period": 12, "session": 2, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 32, "name": "FINC 330",  "credits": 3, "type": "major",    "completed": 0, "period": 12, "session": 1, "prerequisite": "ACCT 221 & STAT 200"             }, 
    { "student_info_id": 10, "seq": 33, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period": 12, "session": 3, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 34, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period": 13, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 35, "name": "BMGT 496",  "credits": 3, "type": "major",    "completed": 0, "period": 13, "session": 2, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 36, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period": 13, "session": 3, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 37, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period": 14, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 38, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period": 14, "session": 2, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 39, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period": 15, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 10, "seq": 40, "name": "BMGT 495",  "credits": 3, "type": "major",    "completed": 0, "period": 15, "session": 2, "prerequisite": "BMGT 365 & MRKT 310 & FINC 330"  }, 
    { "student_info_id": 10, "seq": 41, "name": "CAPSTONE",  "credits": 1, "type": "elective", "completed": 0, "period": 15, "session": 3, "prerequisite": "FINC 330"                        }, 
    { "student_info_id": 3, "seq":  0, "name": "STAT 200",  "credits": 3, "type": "required", "completed": 1, "period":  0, "session": 0, "prerequisite": ""                                },
    { "student_info_id": 3, "seq":  0, "name": "HUMN 100",  "credits": 3, "type": "general",  "completed": 1, "period":  0, "session": 0, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq":  0, "name": "ARTH 334",  "credits": 3, "type": "general",  "completed": 1, "period":  0, "session": 0, "prerequisite": ""                                },
    { "student_info_id": 3, "seq":  0, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 1, "period":  0, "session": 0, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq":  0, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 1, "period":  0, "session": 0, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq":  0, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 1, "period":  0, "session": 0, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq":  0, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 1, "period":  0, "session": 0, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq":  0, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 1, "period":  0, "session": 0, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq":  0, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 1, "period":  0, "session": 0, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq":  0, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 1, "period":  0, "session": 0, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq":  0, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 1, "period":  0, "session": 0, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq":  1, "name": "PACE 111B", "credits": 3, "type": "general",  "completed": 0, "period":  1, "session": 1, "prerequisite": ""                                },
    { "student_info_id": 3, "seq":  2, "name": "LIBS 150",  "credits": 1, "type": "general",  "completed": 0, "period":  1, "session": 2, "prerequisite": ""                                },
    { "student_info_id": 3, "seq":  3, "name": "WRTG 111",  "credits": 3, "type": "general",  "completed": 0, "period":  1, "session": 3, "prerequisite": ""                                },
    { "student_info_id": 3, "seq":  4, "name": "WRTG 112",  "credits": 3, "type": "general",  "completed": 0, "period":  2, "session": 1, "prerequisite": ""                                },
    { "student_info_id": 3, "seq":  5, "name": "NUTR 100",  "credits": 3, "type": "general",  "completed": 0, "period":  2, "session": 2, "prerequisite": ""                                },
    { "student_info_id": 3, "seq":  6, "name": "BMGT 110",  "credits": 3, "type": "major",    "completed": 0, "period":  3, "session": 3, "prerequisite": ""                                },
    { "student_info_id": 3, "seq":  7, "name": "SPCH 100",  "credits": 3, "type": "general",  "completed": 0, "period":  3, "session": 1, "prerequisite": ""                                },
    { "student_info_id": 3, "seq":  8, "name": "IFSM 300",  "credits": 3, "type": "required", "completed": 0, "period":  3, "session": 2, "prerequisite": ""                                },
    { "student_info_id": 3, "seq":  9, "name": "ACCT 220",  "credits": 3, "type": "major",    "completed": 0, "period":  4, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq": 10, "name": "BIOL 103",  "credits": 4, "type": "general",  "completed": 0, "period":  4, "session": 2, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq": 11, "name": "ECON 201",  "credits": 3, "type": "required", "completed": 0, "period":  4, "session": 3, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq": 12, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period":  5, "session": 2, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq": 13, "name": "ECON 203",  "credits": 3, "type": "required", "completed": 0, "period":  5, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq": 14, "name": "ACCT 221",  "credits": 3, "type": "major",    "completed": 0, "period":  5, "session": 3, "prerequisite": "ACCT 220"                        }, 
    { "student_info_id": 3, "seq": 15, "name": "BMGT 364",  "credits": 3, "type": "major",    "completed": 0, "period":  6, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq": 16, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period":  6, "session": 2, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq": 17, "name": "BMGT 365",  "credits": 3, "type": "major",    "completed": 0, "period":  7, "session": 1, "prerequisite": "BMGT 364"                        }, 
    { "student_info_id": 3, "seq": 18, "name": "MRKT 310",  "credits": 3, "type": "major",    "completed": 0, "period":  7, "session": 3, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq": 19, "name": "WRTG 394",  "credits": 3, "type": "general",  "completed": 0, "period":  7, "session": 2, "prerequisite": "WRTG 112"                        }, 
    { "student_info_id": 3, "seq": 20, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period":  8, "session": 3, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq": 21, "name": "BMGT 380",  "credits": 3, "type": "major",    "completed": 0, "period":  8, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq": 22, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period":  8, "session": 2, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq": 23, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period":  9, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq": 24, "name": "HRMN 300",  "credits": 3, "type": "major",    "completed": 0, "period":  9, "session": 2, "prerequisite": ""                                },
    { "student_info_id": 3, "seq": 25, "name": "FINC 330",  "credits": 3, "type": "major",    "completed": 0, "period":  9, "session": 3, "prerequisite": "ACCT 221 & STAT 200"             }, 
    { "student_info_id": 3, "seq": 26, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period": 10, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq": 27, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period": 11, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq": 28, "name": "BMGT 496",  "credits": 3, "type": "major",    "completed": 0, "period": 10, "session": 2, "prerequisite": ""                                }, 
    { "student_info_id": 3, "seq": 29, "name": "BMGT 495",  "credits": 3, "type": "major",    "completed": 0, "period": 11, "session": 2, "prerequisite": "BMGT 365 & MRKT 310 & FINC 330"  }, 
    { "student_info_id": 3, "seq": 30, "name": "CAPSTONE",  "credits": 1, "type": "elective", "completed": 0, "period": 11, "session": 3, "prerequisite": "FINC 330"                        },    
    { "student_info_id": 1, "seq":  1, "name": "PACE 111B", "credits": 3, "type": "general",  "completed": 0, "period":  1, "session": 1, "prerequisite": ""                                },
    { "student_info_id": 1, "seq":  2, "name": "LIBS 150",  "credits": 1, "type": "general",  "completed": 0, "period":  1, "session": 2, "prerequisite": ""                                },
    { "student_info_id": 1, "seq":  3, "name": "WRTG 111",  "credits": 3, "type": "general",  "completed": 0, "period":  1, "session": 3, "prerequisite": ""                                },
    { "student_info_id": 1, "seq":  4, "name": "WRTG 112",  "credits": 3, "type": "general",  "completed": 0, "period":  2, "session": 1, "prerequisite": ""                                },
    { "student_info_id": 1, "seq":  5, "name": "NUTR 100",  "credits": 3, "type": "general",  "completed": 0, "period":  2, "session": 2, "prerequisite": ""                                },
    { "student_info_id": 1, "seq":  6, "name": "BMGT 110",  "credits": 3, "type": "major",    "completed": 0, "period":  3, "session": 3, "prerequisite": ""                                },
    { "student_info_id": 1, "seq":  7, "name": "SPCH 100",  "credits": 3, "type": "general",  "completed": 0, "period":  3, "session": 1, "prerequisite": ""                                },
    { "student_info_id": 1, "seq":  8, "name": "STAT 200",  "credits": 3, "type": "required", "completed": 0, "period":  3, "session": 2, "prerequisite": ""                                },
    { "student_info_id": 1, "seq":  9, "name": "IFSM 300",  "credits": 3, "type": "required", "completed": 0, "period":  4, "session": 1, "prerequisite": ""                                },
    { "student_info_id": 1, "seq": 10, "name": "ACCT 220",  "credits": 3, "type": "major",    "completed": 0, "period":  4, "session": 2, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 11, "name": "HUMN 100",  "credits": 3, "type": "general",  "completed": 0, "period":  4, "session": 3, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 12, "name": "BIOL 103",  "credits": 4, "type": "general",  "completed": 0, "period":  5, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 13, "name": "ECON 201",  "credits": 3, "type": "required", "completed": 0, "period":  5, "session": 3, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 14, "name": "ARTH 334",  "credits": 3, "type": "general",  "completed": 0, "period":  5, "session": 2, "prerequisite": ""                                },
    { "student_info_id": 1, "seq": 15, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period":  6, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 16, "name": "ECON 203",  "credits": 3, "type": "required", "completed": 0, "period":  6, "session": 2, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 17, "name": "ACCT 221",  "credits": 3, "type": "major",    "completed": 0, "period":  7, "session": 1, "prerequisite": "ACCT 220"                        }, 
    { "student_info_id": 1, "seq": 18, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period":  7, "session": 2, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 19, "name": "BMGT 364",  "credits": 3, "type": "major",    "completed": 0, "period":  7, "session": 3, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 20, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period":  8, "session": 2, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 21, "name": "BMGT 365",  "credits": 3, "type": "major",    "completed": 0, "period":  8, "session": 1, "prerequisite": "BMGT 364"                        }, 
    { "student_info_id": 1, "seq": 22, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period":  9, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 23, "name": "MRKT 310",  "credits": 3, "type": "major",    "completed": 0, "period":  8, "session": 3, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 24, "name": "WRTG 394",  "credits": 3, "type": "general",  "completed": 0, "period":  9, "session": 2, "prerequisite": "WRTG 112"                        }, 
    { "student_info_id": 1, "seq": 25, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period":  9, "session": 3, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 26, "name": "BMGT 380",  "credits": 3, "type": "major",    "completed": 0, "period": 10, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 27, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period": 10, "session": 2, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 28, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period": 11, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 29, "name": "HRMN 300",  "credits": 3, "type": "major",    "completed": 0, "period": 11, "session": 2, "prerequisite": ""                                },
    { "student_info_id": 1, "seq": 30, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period": 11, "session": 3, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 31, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period": 12, "session": 2, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 32, "name": "FINC 330",  "credits": 3, "type": "major",    "completed": 0, "period": 12, "session": 1, "prerequisite": "ACCT 221 & STAT 200"             }, 
    { "student_info_id": 1, "seq": 33, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period": 12, "session": 3, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 34, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period": 13, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 35, "name": "BMGT 496",  "credits": 3, "type": "major",    "completed": 0, "period": 13, "session": 2, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 36, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period": 13, "session": 3, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 37, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period": 14, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 38, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period": 14, "session": 2, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 39, "name": "ELECTIVE",  "credits": 3, "type": "elective", "completed": 0, "period": 15, "session": 1, "prerequisite": ""                                }, 
    { "student_info_id": 1, "seq": 40, "name": "BMGT 495",  "credits": 3, "type": "major",    "completed": 0, "period": 15, "session": 2, "prerequisite": "BMGT 365 & MRKT 310 & FINC 330"  }, 
    { "student_info_id": 1, "seq": 41, "name": "CAPSTONE",  "credits": 1, "type": "elective", "completed": 0, "period": 15, "session": 3, "prerequisite": "FINC 330"                        }, 
]

drop_table('student_progress',c)
c.execute('''
    CREATE TABLE student_progress (
        id INTEGER PRIMARY KEY,
        student_info_id INTEGER,
        seq INTEGER,
        name TEXT,
        credits INTEGER,
        type TEXT,
        completed INTEGER,
        period INTEGER,
        session INTEGER,
        prerequisite TEXT,
        FOREIGN KEY(student_info_id) REFERENCES student_info(id)
    )
''')

c.executemany('''
    INSERT INTO student_progress (student_info_id, seq, name, credits, type, completed, period, session, prerequisite)
        VALUES (:student_info_id, :seq, :name, :credits, :type, :completed, :period, :session, :prerequisite)
    ''', student_progress)

conn.commit()

In [16]:
################################################################
# menu_areas Table

menu_areas = [
    {"id": 1, "name": "Business & Management"},
    {"id": 2, "name": "Cybersecurity"},
    {"id": 3, "name": "Data Analytics"},
    {"id": 4, "name": "Education & Teaching"},
    {"id": 5, "name": "Healthcare & Science"},
    {"id": 6, "name": "IT & Computer Science"},
    {"id": 7, "name": "Liberal Arts & Communications"},
    {"id": 8, "name": "Public Safety"},
]

drop_table('menu_areas',c)
c.execute('''
    CREATE TABLE menu_areas (
        id INTEGER PRIMARY KEY,
        name TEXT
    )
''')

c.executemany('INSERT INTO menu_areas VALUES (:id, :name)', menu_areas)
conn.commit()

In [17]:
################################################################
# menu_degrees Table

menu_degrees = [
    { "id": 1, "name": "Associate" },
    { "id": 2, "name": "Bachelor's" },
    { "id": 3, "name": "Master's" },
    { "id": 4, "name": "Doctorate" },
    { "id": 5, "name": "Undergraduate Certificate" },
    { "id": 6, "name": "Graduate Certificate "}
]

drop_table('menu_degrees',c)
c.execute('''
    CREATE TABLE menu_degrees (
        id INTEGER PRIMARY KEY,
        name TEXT
    )
''')

c.executemany('INSERT INTO menu_degrees VALUES (:id, :name)', menu_degrees)
conn.commit()

In [18]:
################################################################
# "Menu Programs by Areas" Table

menu_programs_by_areas = [
    {"menu_degree_id": 1, "menu_area_id": 7, "program_id": 1},
    {"menu_degree_id": 2, "menu_area_id": 1, "program_id": 2},
    {"menu_degree_id": 2, "menu_area_id": 1, "program_id": 10},
    {"menu_degree_id": 2, "menu_area_id": 1, "program_id": 32},
    {"menu_degree_id": 2, "menu_area_id": 1, "program_id": 47},
    {"menu_degree_id": 2, "menu_area_id": 1, "program_id": 54},
    {"menu_degree_id": 2, "menu_area_id": 1, "program_id": 55},
    {"menu_degree_id": 2, "menu_area_id": 2, "program_id": 19},
    {"menu_degree_id": 2, "menu_area_id": 2, "program_id": 20},
    {"menu_degree_id": 2, "menu_area_id": 3, "program_id": 21},
    {"menu_degree_id": 2, "menu_area_id": 5, "program_id": 9},
    {"menu_degree_id": 2, "menu_area_id": 5, "program_id": 30},
    {"menu_degree_id": 2, "menu_area_id": 5, "program_id": 37},
    {"menu_degree_id": 2, "menu_area_id": 5, "program_id": 40},
    {"menu_degree_id": 2, "menu_area_id": 5, "program_id": 49},
    {"menu_degree_id": 2, "menu_area_id": 5, "program_id": 59},
    {"menu_degree_id": 2, "menu_area_id": 5, "program_id": 64},    
    {"menu_degree_id": 2, "menu_area_id": 5, "program_id": 69},
    {"menu_degree_id": 2, "menu_area_id": 6, "program_id": 5},
    {"menu_degree_id": 2, "menu_area_id": 6, "program_id": 14},
    {"menu_degree_id": 2, "menu_area_id": 6, "program_id": 20},
    {"menu_degree_id": 2, "menu_area_id": 6, "program_id": 52},
    {"menu_degree_id": 2, "menu_area_id": 6, "program_id": 71},
    {"menu_degree_id": 2, "menu_area_id": 6, "program_id": 74},
    {"menu_degree_id": 2, "menu_area_id": 7, "program_id": 12},
    {"menu_degree_id": 2, "menu_area_id": 7, "program_id": 24},
    {"menu_degree_id": 2, "menu_area_id": 7, "program_id": 28},
    {"menu_degree_id": 2, "menu_area_id": 7, "program_id": 36},
    {"menu_degree_id": 2, "menu_area_id": 7, "program_id": 39},
    {"menu_degree_id": 2, "menu_area_id": 7, "program_id": 42},
    {"menu_degree_id": 2, "menu_area_id": 7, "program_id": 46},
    {"menu_degree_id": 2, "menu_area_id": 8, "program_id": 16},
    {"menu_degree_id": 2, "menu_area_id": 8, "program_id": 44},
    {"menu_degree_id": 2, "menu_area_id": 8, "program_id": 51},
    {"menu_degree_id": 2, "menu_area_id": 8, "program_id": 66},
    {"menu_degree_id": 3, "menu_area_id": 1, "program_id": 77},
    {"menu_degree_id": 3, "menu_area_id": 1, "program_id": 78},
    {"menu_degree_id": 3, "menu_area_id": 1, "program_id": 83},
    {"menu_degree_id": 3, "menu_area_id": 1, "program_id": 85},
    {"menu_degree_id": 3, "menu_area_id": 1, "program_id": 103},
    {"menu_degree_id": 3, "menu_area_id": 1, "program_id": 104},
    {"menu_degree_id": 3, "menu_area_id": 1, "program_id": 105},
    {"menu_degree_id": 3, "menu_area_id": 1, "program_id": 106},
    {"menu_degree_id": 3, "menu_area_id": 1, "program_id": 107},
    {"menu_degree_id": 3, "menu_area_id": 1, "program_id": 108},
    {"menu_degree_id": 3, "menu_area_id": 1, "program_id": 109},
    {"menu_degree_id": 3, "menu_area_id": 1, "program_id": 110},
    {"menu_degree_id": 3, "menu_area_id": 1, "program_id": 111},
    {"menu_degree_id": 3, "menu_area_id": 1, "program_id": 112},
    {"menu_degree_id": 3, "menu_area_id": 1, "program_id": 113},
    {"menu_degree_id": 3, "menu_area_id": 1, "program_id": 114},
    {"menu_degree_id": 3, "menu_area_id": 1, "program_id": 117},
    {"menu_degree_id": 3, "menu_area_id": 2, "program_id": 84},
    {"menu_degree_id": 3, "menu_area_id": 2, "program_id": 86},
    {"menu_degree_id": 3, "menu_area_id": 2, "program_id": 87},
    {"menu_degree_id": 3, "menu_area_id": 2, "program_id": 88},
    {"menu_degree_id": 3, "menu_area_id": 2, "program_id": 90},
    {"menu_degree_id": 3, "menu_area_id": 2, "program_id": 97},
    {"menu_degree_id": 3, "menu_area_id": 3, "program_id": 89},
    {"menu_degree_id": 3, "menu_area_id": 3, "program_id": 94},
    {"menu_degree_id": 3, "menu_area_id": 4, "program_id": 91},
    {"menu_degree_id": 3, "menu_area_id": 4, "program_id": 101},
    {"menu_degree_id": 3, "menu_area_id": 4, "program_id": 102},
    {"menu_degree_id": 3, "menu_area_id": 4, "program_id": 116},
    {"menu_degree_id": 3, "menu_area_id": 5, "program_id": 79},
    {"menu_degree_id": 3, "menu_area_id": 5, "program_id": 80},
    {"menu_degree_id": 3, "menu_area_id": 5, "program_id": 81},
    {"menu_degree_id": 3, "menu_area_id": 5, "program_id": 82},
    {"menu_degree_id": 3, "menu_area_id": 5, "program_id": 92},
    {"menu_degree_id": 3, "menu_area_id": 5, "program_id": 93},
    {"menu_degree_id": 3, "menu_area_id": 6, "program_id": 84},
    {"menu_degree_id": 3, "menu_area_id": 6, "program_id": 94},
    {"menu_degree_id": 3, "menu_area_id": 6, "program_id": 95},
    {"menu_degree_id": 3, "menu_area_id": 6, "program_id": 96},
    {"menu_degree_id": 3, "menu_area_id": 6, "program_id": 97},
    {"menu_degree_id": 3, "menu_area_id": 6, "program_id": 99},
    {"menu_degree_id": 3, "menu_area_id": 6, "program_id": 100},
    {"menu_degree_id": 3, "menu_area_id": 7, "program_id": 115},
    {"menu_degree_id": 3, "menu_area_id": 8, "program_id": 107},
    {"menu_degree_id": 3, "menu_area_id": 8, "program_id": 110},
    {"menu_degree_id": 4, "menu_area_id": 1, "program_id": 118},
    {"menu_degree_id": 4, "menu_area_id": 1, "program_id": 119},
    {"menu_degree_id": 5, "menu_area_id": 1, "program_id": 120},
    {"menu_degree_id": 5, "menu_area_id": 1, "program_id": 121},
    {"menu_degree_id": 5, "menu_area_id": 1, "program_id": 129},
    {"menu_degree_id": 5, "menu_area_id": 1, "program_id": 131},
    {"menu_degree_id": 5, "menu_area_id": 1, "program_id": 133},
    {"menu_degree_id": 5, "menu_area_id": 1, "program_id": 134},
    {"menu_degree_id": 5, "menu_area_id": 1, "program_id": 135},
    {"menu_degree_id": 5, "menu_area_id": 1, "program_id": 137},
    {"menu_degree_id": 5, "menu_area_id": 1, "program_id": 139},
    {"menu_degree_id": 5, "menu_area_id": 2, "program_id": 127},
    {"menu_degree_id": 5, "menu_area_id": 3, "program_id": 128},
    {"menu_degree_id": 5, "menu_area_id": 3, "program_id": 136},
    {"menu_degree_id": 5, "menu_area_id": 5, "program_id": 123},
    {"menu_degree_id": 5, "menu_area_id": 5, "program_id": 125},
    {"menu_degree_id": 5, "menu_area_id": 5, "program_id": 132},
    {"menu_degree_id": 5, "menu_area_id": 5, "program_id": 143},
    {"menu_degree_id": 5, "menu_area_id": 6, "program_id": 124},
    {"menu_degree_id": 5, "menu_area_id": 6, "program_id": 126},
    {"menu_degree_id": 5, "menu_area_id": 6, "program_id": 127},
    {"menu_degree_id": 5, "menu_area_id": 6, "program_id": 130},
    {"menu_degree_id": 5, "menu_area_id": 6, "program_id": 138},
    {"menu_degree_id": 5, "menu_area_id": 6, "program_id": 142},
    {"menu_degree_id": 5, "menu_area_id": 6, "program_id": 144},
    {"menu_degree_id": 5, "menu_area_id": 7, "program_id": 122},
    {"menu_degree_id": 5, "menu_area_id": 7, "program_id": 141},
    {"menu_degree_id": 5, "menu_area_id": 7, "program_id": 145},
    {"menu_degree_id": 5, "menu_area_id": 8, "program_id": 140},
    {"menu_degree_id": 6, "menu_area_id": 1, "program_id": 146},
    {"menu_degree_id": 6, "menu_area_id": 1, "program_id": 147},
    {"menu_degree_id": 6, "menu_area_id": 1, "program_id": 161},
    {"menu_degree_id": 6, "menu_area_id": 1, "program_id": 164},
    {"menu_degree_id": 6, "menu_area_id": 1, "program_id": 165},
    {"menu_degree_id": 6, "menu_area_id": 1, "program_id": 167},
    {"menu_degree_id": 6, "menu_area_id": 2, "program_id": 150},
    {"menu_degree_id": 6, "menu_area_id": 2, "program_id": 151},
    {"menu_degree_id": 6, "menu_area_id": 2, "program_id": 154},
    {"menu_degree_id": 6, "menu_area_id": 2, "program_id": 159},
    {"menu_degree_id": 6, "menu_area_id": 3, "program_id": 149},
    {"menu_degree_id": 6, "menu_area_id": 4, "program_id": 160},
    {"menu_degree_id": 6, "menu_area_id": 4, "program_id": 162},
    {"menu_degree_id": 6, "menu_area_id": 5, "program_id": 155},
    {"menu_degree_id": 6, "menu_area_id": 5, "program_id": 156},
    {"menu_degree_id": 6, "menu_area_id": 5, "program_id": 163},
    {"menu_degree_id": 6, "menu_area_id": 6, "program_id": 158},
    {"menu_degree_id": 6, "menu_area_id": 6, "program_id": 168},
    {"menu_degree_id": 6, "menu_area_id": 7, "program_id": 166}
]

drop_table('menu_programs_by_areas',c)
c.execute('''
    CREATE TABLE menu_programs_by_areas (
        id INTEGER PRIMARY KEY,
        menu_area_id INTEGER,
        menu_degree_id INTEGER,
        program_id INTEGER,
        FOREIGN KEY(menu_area_id) REFERENCES menu_areas(id)
        FOREIGN KEY(menu_degree_id) REFERENCES menu_degrees(id)
        FOREIGN KEY(program_id) REFERENCES programs(id)
    )
''')

c.executemany('''
    INSERT INTO menu_programs_by_areas (menu_area_id, menu_degree_id, program_id) 
        VALUES (:menu_area_id, :menu_degree_id, :program_id)
''', menu_programs_by_areas)
conn.commit()

In [ ]:
# Close the connection
conn.close()